In [8]:
pip install aiohttp

Note: you may need to restart the kernel to use updated packages.


In [4]:
tiers=["IRON","BRONZE","SILVER","GOLD","PLATINUM","EMERALD","DIAMOND"]
divisions = ["I", "II", "III", "IV"]
pages=["1","2"]

In [ ]:
#https://developer.riotgames.com/apis#summoner-v4/GET_getBySummonerName
#https://developer.riotgames.com/apis#match-v5/GET_getMatchIdsByPUUID

In [5]:
import csv
import requests
import time  # Import the time module

def fetch_and_save_ranked_players(tiers, divisions, pages):
    api_key = "RGAPI-8840c4e8-ef3d-40cb-90fa-2ea005f2bb1c"  # Use your actual API key
    base_url = "https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{tier}/{division}?page={page}"
    headers = {
        "X-Riot-Token": api_key
    }
    
    file_path = '../DataProcessing/TestData/rankedPlayersKorea.csv'
    
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['tier', 'division', 'summonerName'])  # CSV Header

        for tier in tiers:
            for division in divisions:
                for page in pages:
                    url = base_url.format(tier=tier, division=division, page=page)
                    response = requests.get(url, headers=headers)

                    # Wait for 2 seconds before making the next request
                    time.sleep(2)

                    if response.status_code == 200:
                        players = response.json()
                        for player in players:
                            if player['wins'] + player['losses'] > 20:
                                writer.writerow([player['tier'], player['rank'], player['summonerName']])
                    else:
                        print(f"Error: {response.status_code}, {response.text}")
# Note: Before running, replace 'Your_API_Key_Here' with your actual API key and ensure the file path for CSV is correct as per your environment.


In [6]:
#ONLY RUN IF NEW DATA REQUIRED
fetch_and_save_ranked_players(tiers, divisions, pages)

In [9]:
import csv

def find_duplicate_summoner_names_with_rows(file_path):
    summoner_occurrences = {}  # Tracks summoner names and their row numbers
    duplicates_info = []  # Stores information about duplicates

    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row_number, row in enumerate(reader, start=1):  # Start counting rows from 1
            summoner_name = row['summonerName']
            if summoner_name in summoner_occurrences:
                # If the summoner name is already encountered, add the current row as a duplicate
                summoner_occurrences[summoner_name].append(row_number)
            else:
                # Otherwise, initialize the list with the current row number
                summoner_occurrences[summoner_name] = [row_number]

    # Filter out summoner names with more than one occurrence and prepare duplicate info
    for summoner_name, row_numbers in summoner_occurrences.items():
        if len(row_numbers) > 1:
            duplicates_info.append((summoner_name, row_numbers))

    return duplicates_info

# Specify the file path
file_path = '../DataProcessing/TestData/rankedPlayersKorea.csv'
duplicates = find_duplicate_summoner_names_with_rows(file_path)

# Example output
for summoner_name, rows in duplicates:
    print(f"Duplicate: {summoner_name} found in rows: {rows}")


Duplicate: 미라클제이티 found in rows: [67, 70]
Duplicate: KoreaJy found in rows: [68, 71]
Duplicate: 비챤 팬인 고라니 found in rows: [69, 72]
Duplicate: 무적대봉 found in rows: [225, 228]
Duplicate: Mute All Live found in rows: [226, 229]
Duplicate: 아개졸 found in rows: [227, 230]
Duplicate: 핸젤과그랬디야 found in rows: [381, 384]
Duplicate: 100363476del found in rows: [382, 385]
Duplicate: 오리새뀌 found in rows: [383, 386]
Duplicate: 삐뚤어진 또치 found in rows: [632, 634]
Duplicate: 껍질벗기기기장인 found in rows: [633, 635]
Duplicate: 기운찬곰텡 found in rows: [869, 874]
Duplicate: 미드보짓군영 found in rows: [870, 875]
Duplicate: Future maker KR found in rows: [871, 876]
Duplicate: 응급실카사노바 found in rows: [872, 877]
Duplicate: PAKACHU6 found in rows: [873, 878]
Duplicate: 백만송이라벤더 found in rows: [1071, 1073]
Duplicate: 꾸그밍 found in rows: [1072, 1074]
Duplicate: 간티드 found in rows: [1267, 1268]
Duplicate: XX황족미드XX found in rows: [1469, 1471]
Duplicate: 피키포킹 found in rows: [1470, 1472]
Duplicate: 우주의치킨 found in rows: [1679, 1683]
Duplica

In [10]:
import csv

def remove_duplicate_summoner_names(file_path):
    processed_summoners = set()  # To track unique summonerNames
    unique_rows = []  # To store rows after removing duplicates

    # Read the file and filter out duplicate summonerNames
    with open(file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            if row['summonerName'] not in processed_summoners:
                processed_summoners.add(row['summonerName'])
                unique_rows.append(row)

    # Write the unique rows back to the file
    with open(file_path, mode='w', newline='', encoding='utf-8') as outfile:
        fieldnames = ['tier', 'division', 'summonerName']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(unique_rows)

# Specify the file path
#file_path = '../DataProcessing/TestData/rankedPlayers.csv'
remove_duplicate_summoner_names(file_path)


In [11]:
import os
import csv

def split_csv_by_iron_tier(source_file_path):
    # Define the base folder for the new files
    base_folder = os.path.dirname(source_file_path)
    
    # Initialize a dictionary to keep track of the CSV writer for the GOLD tier
    writer = None
    file = None
    
    with open(source_file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            tier = row['tier']
            
            # Proceed only if the tier is GOLD
            if tier == "IRON":
                file_name = "IRON_players_korea.csv"
                file_path = os.path.join(base_folder, file_name)
                
                # Check if we already have a writer for the GOLD tier, if not, create it
                if writer is None:
                    file = open(file_path, mode='w', newline='', encoding='utf-8')
                    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
                    writer.writeheader()
                
                # Write the row to the GOLD file
                writer.writerow(row)
    
    # Close the file for the GOLD tier if it's open
    if file is not None:
        file.close()

# Specify the source file path
source_file_path = '../DataProcessing/TestData/rankedPlayersKorea.csv'

# Call the function
split_csv_by_iron_tier(source_file_path)


In [12]:
import os
import csv

def split_csv_by_bronze_tier(source_file_path):
    # Define the base folder for the new files
    base_folder = os.path.dirname(source_file_path)
    
    # Initialize a dictionary to keep track of the CSV writer for the GOLD tier
    writer = None
    file = None
    
    with open(source_file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            tier = row['tier']
            
            # Proceed only if the tier is GOLD
            if tier == "BRONZE":
                file_name = "BRONZE_players_korea.csv"
                file_path = os.path.join(base_folder, file_name)
                
                # Check if we already have a writer for the GOLD tier, if not, create it
                if writer is None:
                    file = open(file_path, mode='w', newline='', encoding='utf-8')
                    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
                    writer.writeheader()
                
                # Write the row to the GOLD file
                writer.writerow(row)
    
    # Close the file for the GOLD tier if it's open
    if file is not None:
        file.close()

# Specify the source file path
source_file_path = '../DataProcessing/TestData/rankedPlayersKorea.csv'

# Call the function
split_csv_by_bronze_tier(source_file_path)


In [13]:
import os
import csv

def split_csv_by_silver_tier(source_file_path):
    # Define the base folder for the new files
    base_folder = os.path.dirname(source_file_path)
    
    # Initialize a dictionary to keep track of the CSV writer for the GOLD tier
    writer = None
    file = None
    
    with open(source_file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            tier = row['tier']
            
            # Proceed only if the tier is GOLD
            if tier == "SILVER":
                file_name = "SILVER_players_korea.csv"
                file_path = os.path.join(base_folder, file_name)
                
                # Check if we already have a writer for the GOLD tier, if not, create it
                if writer is None:
                    file = open(file_path, mode='w', newline='', encoding='utf-8')
                    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
                    writer.writeheader()
                
                # Write the row to the GOLD file
                writer.writerow(row)
    
    # Close the file for the GOLD tier if it's open
    if file is not None:
        file.close()

# Specify the source file path
source_file_path = '../DataProcessing/TestData/rankedPlayersKorea.csv'

# Call the function
split_csv_by_silver_tier(source_file_path)

In [14]:
import os
import csv

def split_csv_by_gold_tier(source_file_path):
    # Define the base folder for the new files
    base_folder = os.path.dirname(source_file_path)
    
    # Initialize a dictionary to keep track of the CSV writer for the GOLD tier
    writer = None
    file = None
    
    with open(source_file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            tier = row['tier']
            
            # Proceed only if the tier is GOLD
            if tier == "GOLD":
                file_name = "GOLD_players_korea.csv"
                file_path = os.path.join(base_folder, file_name)
                
                # Check if we already have a writer for the GOLD tier, if not, create it
                if writer is None:
                    file = open(file_path, mode='w', newline='', encoding='utf-8')
                    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
                    writer.writeheader()
                
                # Write the row to the GOLD file
                writer.writerow(row)
    
    # Close the file for the GOLD tier if it's open
    if file is not None:
        file.close()

# Specify the source file path
source_file_path = '../DataProcessing/TestData/rankedPlayersKorea.csv'

# Call the function
split_csv_by_gold_tier(source_file_path)

In [15]:
import os
import csv

def split_csv_by_platinum_tier(source_file_path):
    # Define the base folder for the new files
    base_folder = os.path.dirname(source_file_path)
    
    # Initialize a dictionary to keep track of the CSV writer for the GOLD tier
    writer = None
    file = None
    
    with open(source_file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            tier = row['tier']
            
            # Proceed only if the tier is GOLD
            if tier == "PLATINUM":
                file_name = "PLATINUM_players_korea.csv"
                file_path = os.path.join(base_folder, file_name)
                
                # Check if we already have a writer for the GOLD tier, if not, create it
                if writer is None:
                    file = open(file_path, mode='w', newline='', encoding='utf-8')
                    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
                    writer.writeheader()
                
                # Write the row to the GOLD file
                writer.writerow(row)
    
    # Close the file for the GOLD tier if it's open
    if file is not None:
        file.close()

# Specify the source file path
source_file_path = '../DataProcessing/TestData/rankedPlayersKorea.csv'

# Call the function
split_csv_by_platinum_tier(source_file_path)

In [16]:
import os
import csv

def split_csv_by_emerald_tier(source_file_path):
    # Define the base folder for the new files
    base_folder = os.path.dirname(source_file_path)
    
    # Initialize a dictionary to keep track of the CSV writer for the GOLD tier
    writer = None
    file = None
    
    with open(source_file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            tier = row['tier']
            
            # Proceed only if the tier is GOLD
            if tier == "EMERALD":
                file_name = "EMERALD_players_korea.csv"
                file_path = os.path.join(base_folder, file_name)
                
                # Check if we already have a writer for the GOLD tier, if not, create it
                if writer is None:
                    file = open(file_path, mode='w', newline='', encoding='utf-8')
                    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
                    writer.writeheader()
                
                # Write the row to the GOLD file
                writer.writerow(row)
    
    # Close the file for the GOLD tier if it's open
    if file is not None:
        file.close()

# Specify the source file path
source_file_path = '../DataProcessing/TestData/rankedPlayersKorea.csv'

# Call the function
split_csv_by_emerald_tier(source_file_path)

In [21]:
import os
import csv

def split_csv_by_diamond_tier(source_file_path):
    # Define the base folder for the new files
    base_folder = os.path.dirname(source_file_path)
    
    # Initialize a dictionary to keep track of the CSV writer for the GOLD tier
    writer = None
    file = None
    
    with open(source_file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            tier = row['tier']
            
            # Proceed only if the tier is GOLD
            if tier == "DIAMOND":
                file_name = "DIAMOND_players_korea.csv"
                file_path = os.path.join(base_folder, file_name)
                
                # Check if we already have a writer for the GOLD tier, if not, create it
                if writer is None:
                    file = open(file_path, mode='w', newline='', encoding='utf-8')
                    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
                    writer.writeheader()
                
                # Write the row to the GOLD file
                writer.writerow(row)
    
    # Close the file for the GOLD tier if it's open
    if file is not None:
        file.close()

# Specify the source file path
source_file_path = '../DataProcessing/TestData/rankedPlayersKorea.csv'

# Call the function
split_csv_by_diamond_tier(source_file_path)

In [22]:
def get_summoner_names_from_csv(file_path):
    summoner_names = []
    
    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            summoner_names.append(row['summonerName'])
    
    return summoner_names

# Specify the path to your CSV file
file_path = '../DataProcessing/TestData/DIAMOND_players_korea.csv'
summoner_names = get_summoner_names_from_csv(file_path)
print(summoner_names)


['dnflxlaanjgka', '김지수보고싶다', 'jg umpire', '거 픔', '체승븜', '카리Na', 'came from you', '뚜쓩빠쓩', '강 근 복', '첼까지이렐함', '00년생 야옹이', 'QQ0117', '접습니다123', 'CSEK', 'Dope Shxx', '망상감상대상', 'Raumdoyter', '괴롭히지마피즈', 'Refresh1', '울집강아지는제니', 'Modlus', '칭찬들으면잘해요', 'kimddabu', 'Stellazz', '하이드 온 부씨', '밥이랑 고기', '동김명', '대리기사녹턴', '지브릴6계급', '짜피나크면못막아', '던져드릴게요', 'Sereno', '전설이된취사병', '주성이에염', '때린다 너', '걍주원', '규렁뱅이', '하체튼튼상체부실', 'Swoos', 'Mu Tetsu ho', 'iliiiIIillliIIIl', 'ADO Fans', '누가범인이지', 'dsnm', '지민짱짱143', '가자꾸나 뽀찌야', '찡 벅', '라인전하기싫어', 'Xio Yeqiufeng', '빨간악보', '럭스 냥냥', '멈출수없는인생', '나의이름은박효빈', 'Sacra Les', 'moochin련', '갱장히 도라히', 'Mid Kismet', 'Facey', 'Bajudeva', '미안하다봐주라', '19살 탑솔러', '무관귀1신', '팀운99실력1게임', '평화공원', '궁우스', '구영eeee', '무난한선픽카드', 'heartST', 'Mid Briar', '델루나무나', '백 테', '돌잡이 코인잡음', 'je te regrette', '펜시차일드', '원딜벽느낀사람', '롤개망해라망겜', '조진희', '너진짜죽지좀마라', '요네제이스아트', '간단해요', '밥만 잘먹어요', '지 띵', '채팅끄고줏대있게', '으디 주인을 물어', 'top 마이', '명작위작무', '원딜아 예뻐해줘', '다딱이는원숭이다', '치 몬', 'Jyu Biolle Grace', '김치용', '칸나의 꼬리', 'nnI

In [23]:
import requests
import time
import csv
import os
#FINDS PUUID FROM SUMMONER NAME
def update_summoner_names_with_puuid(file_path, summoner_names):
    api_key = "RGAPI-8840c4e8-ef3d-40cb-90fa-2ea005f2bb1c"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api_key
    }
    summoner_to_puuid = {}

    start_time = time.time()
    last_update_time = start_time
    total_names = len(summoner_names)
    names_processed = 0

    # Fetch puuid for each summoner name
    for name in summoner_names:
        url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}"
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            summoner_to_puuid[name] = data['puuid']
        else:
            print(f"Error fetching data for {name}: {response.status_code}")
        names_processed += 1
        current_time = time.time()

        # Check if 1 minute has passed for progress update
        if current_time - last_update_time >= 60:
            print(f"Progress: {names_processed}/{total_names} summoner names processed.")
            last_update_time = current_time

        time.sleep(1.5)  # Delay to comply with rate limit
    
    # Update progress after finishing all requests
    print(f"Progress: {names_processed}/{total_names} summoner names processed. Update complete.")

    # Now, replace summoner names with puuid in the CSV
    temp_file_path = file_path + ".tmp"
    with open(file_path, mode='r', encoding='utf-8') as infile, open(temp_file_path, mode='w', newline='', encoding='utf-8') as outfile:
        reader = csv.DictReader(infile)
        writer = csv.DictWriter(outfile, fieldnames=reader.fieldnames)
        writer.writeheader()
        for row in reader:
            if row['summonerName'] in summoner_to_puuid:
                row['summonerName'] = summoner_to_puuid[row['summonerName']]
            writer.writerow(row)
    
    # Replace original file with the updated temp file
    os.replace(temp_file_path, file_path)


# Call the function
update_summoner_names_with_puuid(file_path, summoner_names)



Progress: 21/1431 summoner names processed.
Progress: 42/1431 summoner names processed.
Progress: 63/1431 summoner names processed.
Progress: 84/1431 summoner names processed.
Progress: 105/1431 summoner names processed.
Progress: 125/1431 summoner names processed.
Progress: 146/1431 summoner names processed.
Progress: 166/1431 summoner names processed.
Progress: 186/1431 summoner names processed.
Progress: 206/1431 summoner names processed.
Progress: 227/1431 summoner names processed.
Progress: 247/1431 summoner names processed.
Progress: 267/1431 summoner names processed.
Progress: 287/1431 summoner names processed.
Progress: 307/1431 summoner names processed.
Progress: 327/1431 summoner names processed.
Progress: 347/1431 summoner names processed.
Progress: 367/1431 summoner names processed.
Progress: 387/1431 summoner names processed.
Progress: 407/1431 summoner names processed.
Progress: 427/1431 summoner names processed.
Progress: 447/1431 summoner names processed.
Progress: 467/

In [24]:
file_path = '../DataProcessing/TestData/EMERALD_players_korea.csv'
summoner_names = get_summoner_names_from_csv(file_path)
print(summoner_names)
update_summoner_names_with_puuid(file_path, summoner_names)


file_path = '../DataProcessing/TestData/PLATINUM_players_korea.csv'
summoner_names = get_summoner_names_from_csv(file_path)
print(summoner_names)
update_summoner_names_with_puuid(file_path, summoner_names)

file_path = '../DataProcessing/TestData/GOLD_players_korea.csv'
summoner_names = get_summoner_names_from_csv(file_path)
print(summoner_names)
update_summoner_names_with_puuid(file_path, summoner_names)

file_path = '../DataProcessing/TestData/SILVER_players_korea.csv'
summoner_names = get_summoner_names_from_csv(file_path)
print(summoner_names)
update_summoner_names_with_puuid(file_path, summoner_names)

file_path = '../DataProcessing/TestData/BRONZE_players_korea.csv'
summoner_names = get_summoner_names_from_csv(file_path)
print(summoner_names)
update_summoner_names_with_puuid(file_path, summoner_names)

file_path = '../DataProcessing/TestData/IRON_players_korea.csv'
summoner_names = get_summoner_names_from_csv(file_path)
print(summoner_names)
update_summoner_names_with_puuid(file_path, summoner_names)

['막타의호흡', '냥냥와플', '10000229del', '핫율무라떼', '씅 질', '정글몹에죽는사람', '힘이 센', '금강아빠', '김쁘앙99', '유댕냥', '닳           븃', 'SeoBy2ol', '도레미파솔라시아', '담배빤아이번', '호치킨집사장님', '난 증명할것이다', '호시노 기연', '아리취향', '우삼겹짜글이', '고아쓰레기게임', '나죽이지마마', 'cofrksek', '우월감은곧열등감', '강창강', 'Uber', '부캐탑연습', '천안 김민수', '규표에요', '오아시스에서 사막을 왜 찾아', 'rordori', '100142846del', '한준공주님', '뽐 여름 괄 겨울', '삭제 베이가', '돈땃지', '연습용아이디97', '쏭카페01', '용빔3', '쓰레기놈', '달려볼께 나봐줘', 'agony1', 'JUNOKlNG', '상식개변남', '하이롱롱', 'TWIST', '진짜 탑탑하네', '승패정하는사람', 'Trap flying', '100201654del', '안달체', '쵸신성', 'praz', '왜상대서폿만잘해', 'HEAD ABOVE WATER', 'nuclear99', '태풍공조 부사장', '라니는 못 말려', '2S4N', 'OOOO0O', '민니 공듀', 'C oйt r o L', '호 퀸', '김우잔', '랄부탱탱랄부', 'l구라야구라야l', '쉽가눙', '그냥게임하는인간', '쓰레쉬레쓰', '애니아빠티버지', 'asrfadfsafsd', '항상정글이못해', '우리탑뭐하는뒈', 'lololoololool', '김 나삐', '한잔술에잊자그만나도숨좀쉬자제발', 'IlIIIlIlIIIlIIII', '말걸지마라잡종', '복실구름', '5코3성람머스', '억까하면 오픈함', '신속배달서비스', '서경주', 'seyxxn', '탑은EZ한라인', '엉덩장인이엉덩', '나경누나좋아해요', '프란시스 문가누', '1선 선다', '늙쇠한피지컬', '짱 떡', 'EBS 카타 강사', '꼉서야', '의문의 샤코장인',

Progress: 21/1110 summoner names processed.
Progress: 41/1110 summoner names processed.
Progress: 61/1110 summoner names processed.
Progress: 81/1110 summoner names processed.
Progress: 101/1110 summoner names processed.
Progress: 121/1110 summoner names processed.
Progress: 142/1110 summoner names processed.
Progress: 162/1110 summoner names processed.
Progress: 182/1110 summoner names processed.
Progress: 203/1110 summoner names processed.
Progress: 224/1110 summoner names processed.
Progress: 244/1110 summoner names processed.
Progress: 265/1110 summoner names processed.
Progress: 285/1110 summoner names processed.
Progress: 305/1110 summoner names processed.
Progress: 325/1110 summoner names processed.
Progress: 345/1110 summoner names processed.
Progress: 365/1110 summoner names processed.
Progress: 385/1110 summoner names processed.
Progress: 405/1110 summoner names processed.
Progress: 426/1110 summoner names processed.
Progress: 446/1110 summoner names processed.
Progress: 466/

Progress: 21/964 summoner names processed.
Progress: 42/964 summoner names processed.
Progress: 62/964 summoner names processed.
Progress: 82/964 summoner names processed.
Progress: 102/964 summoner names processed.
Progress: 122/964 summoner names processed.
Progress: 142/964 summoner names processed.
Progress: 162/964 summoner names processed.
Progress: 182/964 summoner names processed.
Progress: 202/964 summoner names processed.
Progress: 223/964 summoner names processed.
Progress: 244/964 summoner names processed.
Progress: 265/964 summoner names processed.
Progress: 285/964 summoner names processed.
Progress: 306/964 summoner names processed.
Progress: 326/964 summoner names processed.
Progress: 347/964 summoner names processed.
Progress: 367/964 summoner names processed.
Progress: 387/964 summoner names processed.
Progress: 407/964 summoner names processed.
Progress: 428/964 summoner names processed.
Progress: 449/964 summoner names processed.
Progress: 470/964 summoner names pro

Progress: 21/843 summoner names processed.
Progress: 41/843 summoner names processed.
Progress: 61/843 summoner names processed.
Progress: 81/843 summoner names processed.
Progress: 101/843 summoner names processed.
Progress: 121/843 summoner names processed.
Progress: 141/843 summoner names processed.
Progress: 161/843 summoner names processed.
Progress: 182/843 summoner names processed.
Progress: 203/843 summoner names processed.
Progress: 223/843 summoner names processed.
Progress: 244/843 summoner names processed.
Progress: 265/843 summoner names processed.
Progress: 285/843 summoner names processed.
Progress: 305/843 summoner names processed.
Progress: 325/843 summoner names processed.
Progress: 345/843 summoner names processed.
Progress: 365/843 summoner names processed.
Progress: 385/843 summoner names processed.
Progress: 406/843 summoner names processed.
Progress: 426/843 summoner names processed.
Progress: 446/843 summoner names processed.
Progress: 466/843 summoner names pro

Progress: 21/921 summoner names processed.
Progress: 42/921 summoner names processed.
Progress: 62/921 summoner names processed.
Progress: 82/921 summoner names processed.
Progress: 102/921 summoner names processed.
Progress: 123/921 summoner names processed.
Progress: 143/921 summoner names processed.
Progress: 164/921 summoner names processed.
Progress: 184/921 summoner names processed.
Progress: 205/921 summoner names processed.
Progress: 226/921 summoner names processed.
Progress: 246/921 summoner names processed.
Progress: 266/921 summoner names processed.
Progress: 286/921 summoner names processed.
Progress: 307/921 summoner names processed.
Progress: 327/921 summoner names processed.
Progress: 347/921 summoner names processed.
Progress: 367/921 summoner names processed.
Progress: 387/921 summoner names processed.
Progress: 407/921 summoner names processed.
Progress: 428/921 summoner names processed.
Progress: 448/921 summoner names processed.
Progress: 468/921 summoner names pro

Progress: 21/774 summoner names processed.
Progress: 41/774 summoner names processed.
Progress: 61/774 summoner names processed.
Progress: 81/774 summoner names processed.
Progress: 101/774 summoner names processed.
Progress: 122/774 summoner names processed.
Progress: 142/774 summoner names processed.
Progress: 162/774 summoner names processed.
Progress: 182/774 summoner names processed.
Progress: 202/774 summoner names processed.
Progress: 223/774 summoner names processed.
Progress: 244/774 summoner names processed.
Progress: 265/774 summoner names processed.
Progress: 285/774 summoner names processed.
Progress: 305/774 summoner names processed.
Progress: 326/774 summoner names processed.
Progress: 347/774 summoner names processed.
Progress: 368/774 summoner names processed.
Progress: 388/774 summoner names processed.
Progress: 408/774 summoner names processed.
Progress: 429/774 summoner names processed.
Progress: 449/774 summoner names processed.
Progress: 469/774 summoner names pro

Progress: 21/762 summoner names processed.
Progress: 41/762 summoner names processed.
Progress: 61/762 summoner names processed.
Progress: 81/762 summoner names processed.
Progress: 102/762 summoner names processed.
Progress: 123/762 summoner names processed.
Progress: 144/762 summoner names processed.
Progress: 164/762 summoner names processed.
Progress: 184/762 summoner names processed.
Progress: 205/762 summoner names processed.
Progress: 225/762 summoner names processed.
Progress: 245/762 summoner names processed.
Progress: 265/762 summoner names processed.
Progress: 286/762 summoner names processed.
Progress: 307/762 summoner names processed.
Progress: 327/762 summoner names processed.
Progress: 348/762 summoner names processed.
Progress: 369/762 summoner names processed.
Progress: 389/762 summoner names processed.
Progress: 410/762 summoner names processed.
Progress: 430/762 summoner names processed.
Progress: 451/762 summoner names processed.
Progress: 472/762 summoner names pro

In [33]:
#det var her jeg havde glemt at tilføje asia istedet for europa i går
import requests
import csv
import time
import os
#Finds matches from puuid
def fetch_matches_and_save(source_csv_path, target_csv_path):
    api_key = "RGAPI-1ec8b3ca-625d-4647-ba76-0b93bf20764d"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api_key
    }
    
    start_time = time.time()
    last_update_time = start_time
    summoners_processed = 0

    # Ensure the target file directory exists
    os.makedirs(os.path.dirname(target_csv_path), exist_ok=True)

    with open(source_csv_path, mode='r', encoding='utf-8') as infile, open(target_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
        csv_reader = csv.DictReader(infile)
        csv_writer = csv.writer(outfile)
        csv_writer.writerow(['MatchID'])  # Header for target CSV
        
        for row in csv_reader:
            puuid = row['summonerName']  # Assuming this column actually contains puuids
            url = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count=20"
            response = requests.get(url, headers=headers)
            
            if response.status_code == 200:
                match_ids = response.json()
                for match_id in match_ids:
                    csv_writer.writerow([match_id])
            else:
                print(f"Error fetching matches for {puuid}: {response.status_code}")
            
            summoners_processed += 1
            current_time = time.time()
            
            # Rate limiting
            time.sleep(1.5)
            
            # Print progress every minute
            if current_time - last_update_time >= 60:
                print(f"Progress: {summoners_processed} summoners processed.")
                last_update_time = current_time

    # Final progress update
    print(f"Final Progress: {summoners_processed} summoners processed. Task complete.")

# Example usage


source_csv_path = '../DataProcessing/TestData/SILVER_players_korea.csv'
target_csv_path = '../DataProcessing/TestData/SILVER_GAMES_KOREA.csv'
fetch_matches_and_save(source_csv_path, target_csv_path)

source_csv_path = '../DataProcessing/TestData/BRONZE_players_korea.csv'
target_csv_path = '../DataProcessing/TestData/BRONZE_GAMES_KOREA.csv'
fetch_matches_and_save(source_csv_path, target_csv_path)

source_csv_path = '../DataProcessing/TestData/IRON_players_korea.csv'
target_csv_path = '../DataProcessing/TestData/IRON_GAMES_KOREA.csv'
fetch_matches_and_save(source_csv_path, target_csv_path)

Progress: 21 summoners processed.
Progress: 41 summoners processed.
Progress: 61 summoners processed.
Progress: 81 summoners processed.
Progress: 101 summoners processed.
Progress: 121 summoners processed.
Progress: 141 summoners processed.
Progress: 161 summoners processed.
Progress: 181 summoners processed.
Progress: 201 summoners processed.
Progress: 221 summoners processed.
Progress: 241 summoners processed.
Progress: 261 summoners processed.
Progress: 281 summoners processed.
Progress: 301 summoners processed.
Progress: 321 summoners processed.
Progress: 341 summoners processed.
Progress: 361 summoners processed.
Progress: 381 summoners processed.
Progress: 401 summoners processed.
Progress: 421 summoners processed.
Progress: 441 summoners processed.
Progress: 461 summoners processed.
Progress: 481 summoners processed.
Progress: 501 summoners processed.
Progress: 521 summoners processed.
Progress: 541 summoners processed.
Progress: 561 summoners processed.
Progress: 581 summoners 

In [36]:
import pandas as pd
def remove_duplicates_and_save(csv_file_path, output_file_path):
    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file_path)
    
    # Remove duplicates based on the 'MatchID' column and keep the first occurrence
    cleaned_df = df.drop_duplicates('MatchID', keep='first')
    
    # Save the cleaned DataFrame to a new CSV file
    cleaned_df.to_csv(output_file_path, index=False)
    
    print(f"Cleaned CSV saved to {output_file_path}.")
    
csv_file_path='../DataProcessing/TestData/IRON_GAMES_KOREA.csv'
output_file_path='../DataProcessing/TestData/CLEANED_IRON_GAMES_KOREA.csv'
remove_duplicates_and_save(csv_file_path, output_file_path)

csv_file_path='../DataProcessing/TestData/BRONZE_GAMES_KOREA.csv'
output_file_path='../DataProcessing/TestData/CLEANED_BRONZE_GAMES_KOREA.csv'
remove_duplicates_and_save(csv_file_path, output_file_path)

csv_file_path='../DataProcessing/TestData/SILVER_GAMES_KOREA.csv'
output_file_path='../DataProcessing/TestData/CLEANED_SILVER_GAMES_KOREA.csv'
remove_duplicates_and_save(csv_file_path, output_file_path)


Cleaned CSV saved to ../DataProcessing/TestData/CLEANED_IRON_GAMES_KOREA.csv.
Cleaned CSV saved to ../DataProcessing/TestData/CLEANED_BRONZE_GAMES_KOREA.csv.
Cleaned CSV saved to ../DataProcessing/TestData/CLEANED_SILVER_GAMES_KOREA.csv.


In [3]:
import csv
import requests
import time
import os
import json

def call_api_and_save_response_with_progress(csv_file_path, save_folder_path):
    # Define the base URL and updated headers for the API call
    base_url = "https://asia.api.riotgames.com/lol/match/v5/matches/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": "RGAPI-e75b46d3-5e51-4829-b978-68790c3ebf56"
    }
    
    # Read Match IDs from the CSV file
    with open(csv_file_path, mode='r', newline='', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        match_ids = [row['MatchID'] for row in csv_reader]
    
    total_matches = len(match_ids)
    matches_processed = 0
    start_time = time.time()
    last_update_time = start_time
    request_times = []  # Track the times of the last 30 requests

    # Process each Match ID
    for match_id in match_ids:
        current_time = time.time()
        
        # Check if we have made 30 requests in the last second
        if len(request_times) >= 30:
            # Calculate the time to wait before making the next request
            time_to_wait = 1 - (current_time - request_times[0])
            if time_to_wait > 0:
                time.sleep(time_to_wait)
        
        full_url = base_url + match_id + "/timeline"
        
        # Make the API call
        response = requests.get(full_url, headers=headers)
        
        # Update the request_times list
        request_times.append(time.time())
        if len(request_times) > 30:
            request_times.pop(0)  # Remove the oldest request time
        
        if response.status_code == 200:
            # Save the response body as a JSON file
            file_path = os.path.join(save_folder_path, f"{match_id}.json")
            with open(file_path, 'w', encoding='utf-8') as json_file:
                json.dump(response.json(), json_file, ensure_ascii=False, indent=4)
        else:
            print(f"Failed to fetch data for Match ID {match_id}. Status Code: {response.status_code}")
        
        matches_processed += 1
        
        if current_time - last_update_time >= 60:
            print(f"Processed {matches_processed}/{total_matches} matches. {int((matches_processed/total_matches)*100)}% complete.")
            last_update_time = current_time
    
    # Final update
    print(f"All {total_matches} matches processed. 100% complete.")

sgamespath='../DataProcessing/TestData/CLEANED_SILVER_GAMES_KOREA.csv'
jsonfilepaths='D:\RawSilverMatchJsonsKorea'
call_api_and_save_response_with_progress(sgamespath, jsonfilepaths)



Processed 24/18273 matches. 0% complete.
Processed 49/18273 matches. 0% complete.
Processed 73/18273 matches. 0% complete.
Processed 98/18273 matches. 0% complete.
Processed 122/18273 matches. 0% complete.
Processed 146/18273 matches. 0% complete.
Processed 168/18273 matches. 0% complete.
Processed 186/18273 matches. 1% complete.
Processed 205/18273 matches. 1% complete.
Processed 226/18273 matches. 1% complete.
Processed 246/18273 matches. 1% complete.
Processed 267/18273 matches. 1% complete.
Processed 288/18273 matches. 1% complete.
Processed 314/18273 matches. 1% complete.
Processed 338/18273 matches. 1% complete.
Processed 363/18273 matches. 1% complete.
Processed 387/18273 matches. 2% complete.
Processed 409/18273 matches. 2% complete.
Processed 430/18273 matches. 2% complete.
Processed 455/18273 matches. 2% complete.
Processed 479/18273 matches. 2% complete.
Processed 503/18273 matches. 2% complete.
Processed 527/18273 matches. 2% complete.
Processed 552/18273 matches. 3% comple

Processed 4263/18273 matches. 23% complete.
Processed 4288/18273 matches. 23% complete.
Processed 4314/18273 matches. 23% complete.
Processed 4338/18273 matches. 23% complete.
Processed 4362/18273 matches. 23% complete.
Processed 4386/18273 matches. 24% complete.
Processed 4410/18273 matches. 24% complete.
Processed 4434/18273 matches. 24% complete.
Processed 4458/18273 matches. 24% complete.
Processed 4482/18273 matches. 24% complete.
Processed 4506/18273 matches. 24% complete.
Processed 4530/18273 matches. 24% complete.
Failed to fetch data for Match ID KR_6911494598. Status Code: 503
Processed 4555/18273 matches. 24% complete.
Processed 4580/18273 matches. 25% complete.
Processed 4604/18273 matches. 25% complete.
Processed 4627/18273 matches. 25% complete.
Processed 4652/18273 matches. 25% complete.
Processed 4677/18273 matches. 25% complete.
Processed 4702/18273 matches. 25% complete.
Processed 4726/18273 matches. 25% complete.
Processed 4749/18273 matches. 25% complete.
Processed 

Processed 8444/18273 matches. 46% complete.
Processed 8470/18273 matches. 46% complete.
Processed 8495/18273 matches. 46% complete.
Processed 8519/18273 matches. 46% complete.
Processed 8544/18273 matches. 46% complete.
Processed 8570/18273 matches. 46% complete.
Processed 8595/18273 matches. 47% complete.
Processed 8620/18273 matches. 47% complete.
Processed 8645/18273 matches. 47% complete.
Failed to fetch data for Match ID KR_6952890226. Status Code: 500
Processed 8670/18273 matches. 47% complete.
Processed 8696/18273 matches. 47% complete.
Processed 8721/18273 matches. 47% complete.
Processed 8746/18273 matches. 47% complete.
Processed 8771/18273 matches. 47% complete.
Processed 8797/18273 matches. 48% complete.
Processed 8823/18273 matches. 48% complete.
Processed 8849/18273 matches. 48% complete.
Processed 8875/18273 matches. 48% complete.
Processed 8902/18273 matches. 48% complete.
Processed 8927/18273 matches. 48% complete.
Processed 8952/18273 matches. 48% complete.
Processed 

Processed 12687/18273 matches. 69% complete.
Processed 12713/18273 matches. 69% complete.
Processed 12738/18273 matches. 69% complete.
Processed 12764/18273 matches. 69% complete.
Processed 12790/18273 matches. 69% complete.
Processed 12817/18273 matches. 70% complete.
Processed 12842/18273 matches. 70% complete.
Processed 12868/18273 matches. 70% complete.
Processed 12894/18273 matches. 70% complete.
Processed 12920/18273 matches. 70% complete.
Processed 12946/18273 matches. 70% complete.
Processed 12972/18273 matches. 70% complete.
Processed 12998/18273 matches. 71% complete.
Processed 13023/18273 matches. 71% complete.
Processed 13048/18273 matches. 71% complete.
Processed 13073/18273 matches. 71% complete.
Processed 13099/18273 matches. 71% complete.
Processed 13124/18273 matches. 71% complete.
Processed 13150/18273 matches. 71% complete.
Processed 13176/18273 matches. 72% complete.
Processed 13202/18273 matches. 72% complete.
Processed 13227/18273 matches. 72% complete.
Processed 

Processed 16956/18273 matches. 92% complete.
Processed 16982/18273 matches. 92% complete.
Processed 17010/18273 matches. 93% complete.
Processed 17037/18273 matches. 93% complete.
Processed 17062/18273 matches. 93% complete.
Processed 17088/18273 matches. 93% complete.
Processed 17113/18273 matches. 93% complete.
Processed 17139/18273 matches. 93% complete.
Processed 17165/18273 matches. 93% complete.
Processed 17190/18273 matches. 94% complete.
Processed 17216/18273 matches. 94% complete.
Processed 17242/18273 matches. 94% complete.
Failed to fetch data for Match ID KR_6951044460. Status Code: 500
Processed 17269/18273 matches. 94% complete.
Processed 17295/18273 matches. 94% complete.
Processed 17322/18273 matches. 94% complete.
Processed 17347/18273 matches. 94% complete.
Processed 17372/18273 matches. 95% complete.
Processed 17399/18273 matches. 95% complete.
Processed 17424/18273 matches. 95% complete.
Processed 17450/18273 matches. 95% complete.
Processed 17476/18273 matches. 95%

In [18]:
import requests
import csv
import time
import os
#Finds matches from puuid
def fetch_matches_and_save(source_csv_path, target_csv_path):
    api_key = "RGAPI-f1974148-4124-4d03-92aa-b1508bcca232"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api_key
    }
    
    start_time = time.time()
    last_update_time = start_time
    summoners_processed = 0

    # Ensure the target file directory exists
    os.makedirs(os.path.dirname(target_csv_path), exist_ok=True)

    with open(source_csv_path, mode='r', encoding='utf-8') as infile, open(target_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
        csv_reader = csv.DictReader(infile)
        csv_writer = csv.writer(outfile)
        csv_writer.writerow(['MatchID'])  # Header for target CSV
        
        for row in csv_reader:
            puuid = row['summonerName']  # Assuming this column actually contains puuids
            url = f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count=20"
            response = requests.get(url, headers=headers)
            
            if response.status_code == 200:
                match_ids = response.json()
                for match_id in match_ids:
                    csv_writer.writerow([match_id])
            else:
                print(f"Error fetching matches for {puuid}: {response.status_code}")
            
            summoners_processed += 1
            current_time = time.time()
            
            # Rate limiting
            time.sleep(1.5)
            
            # Print progress every minute
            if current_time - last_update_time >= 60:
                print(f"Progress: {summoners_processed} summoners processed.")
                last_update_time = current_time

    # Final progress update
    print(f"Final Progress: {summoners_processed} summoners processed. Task complete.")

# Example usage
source_csv_path = '../DataProcessing/TestData/DIAMOND_players.csv'
target_csv_path = '../DataProcessing/TestData/DIAMOND_GAMES.csv'
fetch_matches_and_save(source_csv_path, target_csv_path)


Progress: 32 summoners processed.
Progress: 63 summoners processed.
Progress: 93 summoners processed.
Progress: 124 summoners processed.
Progress: 154 summoners processed.
Progress: 185 summoners processed.
Progress: 216 summoners processed.
Progress: 247 summoners processed.
Progress: 278 summoners processed.
Progress: 309 summoners processed.
Progress: 339 summoners processed.
Progress: 370 summoners processed.
Progress: 401 summoners processed.
Progress: 431 summoners processed.
Progress: 462 summoners processed.
Progress: 493 summoners processed.
Progress: 524 summoners processed.
Progress: 555 summoners processed.
Progress: 586 summoners processed.
Progress: 617 summoners processed.
Progress: 648 summoners processed.
Progress: 679 summoners processed.
Progress: 710 summoners processed.
Progress: 741 summoners processed.
Progress: 771 summoners processed.
Progress: 802 summoners processed.
Progress: 833 summoners processed.
Progress: 864 summoners processed.
Progress: 894 summoners

In [12]:
import asyncio
import aiohttp
import csv
import os
from aiohttp import ClientSession
#FINDS PUUID FROM SUMMONER NAME v2
async def fetch_puuid(session, url, headers, semaphore, summoner_to_puuid, name):
    async with semaphore:
        async with session.get(url, headers=headers) as response:
            if response.status == 200:
                data = await response.json()
                summoner_to_puuid[name] = data['puuid']
            else:
                print(f"Error fetching data for {name}: {response.status}")

async def rate_limited_requester(summoner_names, headers, rate_limit=30):
    tasks = []
    summoner_to_puuid = {}

    semaphore = asyncio.Semaphore(rate_limit)

    async with ClientSession() as session:
        for name in summoner_names:
            url = f"https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}"
            task = asyncio.create_task(fetch_puuid(session, url, headers, semaphore, summoner_to_puuid, name))
            tasks.append(task)

            if len(tasks) % rate_limit == 0:
                await asyncio.sleep(1)

        await asyncio.gather(*tasks)

    return summoner_to_puuid

async def update_summoner_names_with_puuid(file_path, summoner_names):
    api_key = "RGAPI-e75b46d3-5e51-4829-b978-68790c3ebf56"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api_key
    }

    summoner_to_puuid = await rate_limited_requester(summoner_names, headers)

    temp_file_path = file_path + ".tmp"
    with open(file_path, mode='r', encoding='utf-8') as infile, open(temp_file_path, mode='w', newline='', encoding='utf-8') as outfile:
        reader = csv.DictReader(infile)
        writer = csv.DictWriter(outfile, fieldnames=reader.fieldnames)
        writer.writeheader()
        for row in reader:
            if row['summonerName'] in summoner_to_puuid:
                row['summonerName'] = summoner_to_puuid[row['summonerName']]
            writer.writerow(row)
    
    os.replace(temp_file_path, file_path)

# Execution block adapted for Jupyter Notebook
loop = asyncio.get_event_loop()

source_csv_path = "../DataProcessing/TestData/GOLD_players.csv"


if loop.is_running():
    task = loop.create_task(update_summoner_names_with_puuid(source_csv_path, summoner_names))
else:
    loop.run_until_complete(update_summoner_names_with_puuid(source_csv_path, summoner_names))


In [19]:
import asyncio
import aiohttp
import csv
import os
from aiohttp import ClientSession
import datetime

async def fetch_match_ids(session, url, csv_writer, sem, progress):
    async with sem:
        async with session.get(url) as response:
            if response.status == 200:
                match_ids = await response.json()
                for match_id in match_ids:
                    csv_writer.writerow([match_id])
                progress['count'] += 1  # Increment the progress count for each successful fetch
            else:
                print(f"Error: {response.status}")

async def run(source_csv_path, target_csv_path, api_key):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api_key
    }
    
    sem = asyncio.Semaphore(30)  # Control concurrency to 30 requests per second
    progress = {'count': 0, 'last_update': datetime.datetime.now()}  # Initialize progress tracking

    # Ensure the target file directory exists
    os.makedirs(os.path.dirname(target_csv_path), exist_ok=True)

    # Open the target CSV file outside of the async context
    with open(target_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
        csv_writer = csv.writer(outfile)
        csv_writer.writerow(['MatchID'])  # Writing header for match IDs

        async with ClientSession(headers=headers) as session:
            tasks = []
            with open(source_csv_path, mode='r', encoding='utf-8') as infile:
                csv_reader = csv.DictReader(infile)
                for row in csv_reader:
                    puuid = row['summonerName']
                    url = f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count=20"
                    task = asyncio.create_task(fetch_match_ids(session, url, csv_writer, sem, progress))
                    tasks.append(task)
                    
                    # Check and print progress approximately every minute
                    if (datetime.datetime.now() - progress['last_update']).seconds >= 60:
                        print(f"Progress: {progress['count']} summonerNames processed at {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
                        progress['last_update'] = datetime.datetime.now()

                    if len(tasks) >= 30:
                        await asyncio.gather(*tasks)
                        tasks = []  # Reset the list of tasks after processing
                        await asyncio.sleep(1)  # Ensure we don't exceed 30 requests per second

                # Await any remaining tasks
                if tasks:
                    await asyncio.gather(*tasks)

                # Final progress update
                print(f"Final Progress: {progress['count']} summonerNames processed at {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Example usage
source_csv_path = '../DataProcessing/TestData/GOLD_players.csv'
target_csv_path = '../DataProcessing/TestData/GOLD_GAMES.csv'
api_key = "RGAPI-e75b46d3-5e51-4829-b978-68790c3ebf56"

# Adapting to environments with an existing event loop
loop = asyncio.get_event_loop()

if loop.is_running():
    task = loop.create_task(run(source_csv_path, target_csv_path, api_key))
else:
    loop.run_until_complete(run(source_csv_path, target_csv_path, api_key))

    
    
    

Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400
Error: 400

In [25]:
import pandas as pd

def print_duplicates(csv_file_path):
    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file_path)
    
    # Identifying all duplicates, including the first occurrence
    duplicates = df[df.duplicated('MatchID', keep=False)]
    
    # Group by 'MatchID' and aggregate the indexes into a list
    grouped_duplicates = duplicates.groupby('MatchID').apply(lambda x: list(x.index))
    
    # Check if duplicates are found
    if not grouped_duplicates.empty:
        print("Duplicate MatchIDs found along with their locations (indexes):")
        for match_id, indexes in grouped_duplicates.items():
            print(f"MatchID: {match_id}, Indexes: {indexes}")
    else:
        print("No duplicate MatchIDs found.")


In [26]:
print_duplicates(target_csv_path)

Duplicate MatchIDs found along with their locations (indexes):
MatchID: EUW1_6768603417, Indexes: [6052, 25479]
MatchID: EUW1_6772908085, Indexes: [2617, 5918]
MatchID: EUW1_6773073709, Indexes: [11514, 25476]
MatchID: EUW1_6773635054, Indexes: [91, 1264]
MatchID: EUW1_6775918458, Indexes: [23225, 25909]
MatchID: EUW1_6776663774, Indexes: [4323, 20228]
MatchID: EUW1_6776869398, Indexes: [1354, 11112]
MatchID: EUW1_6777574109, Indexes: [15612, 27868]
MatchID: EUW1_6778272331, Indexes: [23600, 26257]
MatchID: EUW1_6778406033, Indexes: [18574, 19979]
MatchID: EUW1_6778799794, Indexes: [16092, 19975]
MatchID: EUW1_6779678725, Indexes: [12135, 20758]
MatchID: EUW1_6781743474, Indexes: [8375, 12131]
MatchID: EUW1_6782855556, Indexes: [8198, 20593]
MatchID: EUW1_6782877199, Indexes: [11729, 15774]
MatchID: EUW1_6784201828, Indexes: [2990, 6995]
MatchID: EUW1_6784555177, Indexes: [15326, 20136]
MatchID: EUW1_6785016306, Indexes: [235, 6066]
MatchID: EUW1_6785113045, Indexes: [4473, 4524]
Match

In [27]:
print(target_csv_path)

../DataProcessing/TestData/DIAMOND_GAMES.csv


In [30]:
def remove_duplicates_and_save(csv_file_path, output_file_path):
    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file_path)
    
    # Remove duplicates based on the 'MatchID' column and keep the first occurrence
    cleaned_df = df.drop_duplicates('MatchID', keep='first')
    
    # Save the cleaned DataFrame to a new CSV file
    cleaned_df.to_csv(output_file_path, index=False)
    
    print(f"Cleaned CSV saved to {output_file_path}.")


In [32]:
csv_file_path='../DataProcessing/TestData/DIAMOND_GAMES.csv'
output_file_path='../DataProcessing/TestData/EUW_DIAMOND_GAMES.csv'
remove_duplicates_and_save(csv_file_path, output_file_path)

Cleaned CSV saved to ../DataProcessing/TestData/EUW_DIAMOND_GAMES.csv.


In [33]:
csv_file_path='../DataProcessing/TestData/EMERALD_GAMES.csv'
output_file_path='../DataProcessing/TestData/EUW_EMERALD_GAMES.csv'
remove_duplicates_and_save(csv_file_path, output_file_path)

Cleaned CSV saved to ../DataProcessing/TestData/EUW_EMERALD_GAMES.csv.


In [34]:
csv_file_path='../DataProcessing/TestData/PLATINUM_GAMES.csv'
output_file_path='../DataProcessing/TestData/EUW_PLATINUM_GAMES.csv'
remove_duplicates_and_save(csv_file_path, output_file_path)

Cleaned CSV saved to ../DataProcessing/TestData/EUW_PLATINUM_GAMES.csv.


In [35]:
csv_file_path='../DataProcessing/TestData/GOLD_GAMES.csv'
output_file_path='../DataProcessing/TestData/EUW_GOLD_GAMES.csv'
remove_duplicates_and_save(csv_file_path, output_file_path)

Cleaned CSV saved to ../DataProcessing/TestData/EUW_GOLD_GAMES.csv.


In [36]:
csv_file_path='../DataProcessing/TestData/SILVER_GAMES.csv'
output_file_path='../DataProcessing/TestData/EUW_SILVER_GAMES.csv'
remove_duplicates_and_save(csv_file_path, output_file_path)

Cleaned CSV saved to ../DataProcessing/TestData/EUW_SILVER_GAMES.csv.


In [37]:
csv_file_path='../DataProcessing/TestData/BRONZE_GAMES.csv'
output_file_path='../DataProcessing/TestData/EUW_BRONZE_GAMES.csv'
remove_duplicates_and_save(csv_file_path, output_file_path)

Cleaned CSV saved to ../DataProcessing/TestData/EUW_BRONZE_GAMES.csv.


In [38]:
csv_file_path='../DataProcessing/TestData/IRON_GAMES.csv'
output_file_path='../DataProcessing/TestData/EUW_IRON_GAMES.csv'
remove_duplicates_and_save(csv_file_path, output_file_path)

Cleaned CSV saved to ../DataProcessing/TestData/EUW_IRON_GAMES.csv.


In [3]:
import csv
import requests
import time
import os
import json

def call_api_and_save_response_with_progress(csv_file_path, save_folder_path):
    # Define the base URL and headers for the API call
    base_url = "https://europe.api.riotgames.com/lol/match/v5/matches/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": "RGAPI-e75b46d3-5e51-4829-b978-68790c3ebf56"
    }
    
    # Read Match IDs from the CSV file
    with open(csv_file_path, mode='r', newline='', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        match_ids = [row['MatchID'] for row in csv_reader]
    
    total_matches = len(match_ids)
    matches_processed = 0
    start_time = time.time()
    last_update_time = start_time

    # Process each Match ID
    for match_id in match_ids:
        full_url = base_url + match_id + "/timeline"
        
        # Make the API call
        response = requests.get(full_url, headers=headers)
        
        if response.status_code == 200:
            # Save the response body as a JSON file
            file_path = os.path.join(save_folder_path, f"{match_id}.json")
            with open(file_path, 'w', encoding='utf-8') as json_file:
                json.dump(response.json(), json_file, ensure_ascii=False, indent=4)
        else:
            print(f"Failed to fetch data for Match ID {match_id}. Status Code: {response.status_code}")
        
        matches_processed += 1
        
        # Respect the API rate limit
        time.sleep(1/30)  # Sleep to ensure not exceeding 30 calls per second
        
        current_time = time.time()
        if current_time - last_update_time >= 60:
            print(f"Processed {matches_processed}/{total_matches} matches. {int((matches_processed/total_matches)*100)}% complete.")
            last_update_time = current_time
    
    # Final update
    print(f"All {total_matches} matches processed. 100% complete.")

bgamespath='../DataProcessing/TestData/BRONZE_GAMES.csv'
jsonfilepath6='D:\RawBronzeMatchJsons'
call_api_and_save_response_with_progress(bgamespath, jsonfilepath6)

Processed 62/13243 matches. 0% complete.
Processed 123/13243 matches. 0% complete.
Processed 186/13243 matches. 1% complete.
Processed 249/13243 matches. 1% complete.
Processed 312/13243 matches. 2% complete.
Processed 374/13243 matches. 2% complete.
Processed 436/13243 matches. 3% complete.
Processed 497/13243 matches. 3% complete.
Processed 561/13243 matches. 4% complete.
Processed 621/13243 matches. 4% complete.
Processed 684/13243 matches. 5% complete.
Processed 747/13243 matches. 5% complete.
Processed 809/13243 matches. 6% complete.
Processed 872/13243 matches. 6% complete.
Processed 933/13243 matches. 7% complete.
Processed 994/13243 matches. 7% complete.
Processed 1057/13243 matches. 7% complete.
Processed 1119/13243 matches. 8% complete.
Processed 1182/13243 matches. 8% complete.
Processed 1244/13243 matches. 9% complete.
Processed 1305/13243 matches. 9% complete.
Processed 1367/13243 matches. 10% complete.
Processed 1427/13243 matches. 10% complete.
Processed 1489/13243 match

Processed 10233/13243 matches. 77% complete.
Processed 10295/13243 matches. 77% complete.
Processed 10357/13243 matches. 78% complete.
Processed 10420/13243 matches. 78% complete.
Processed 10480/13243 matches. 79% complete.
Processed 10544/13243 matches. 79% complete.
Processed 10604/13243 matches. 80% complete.
Processed 10666/13243 matches. 80% complete.
Processed 10728/13243 matches. 81% complete.
Processed 10790/13243 matches. 81% complete.
Processed 10850/13243 matches. 81% complete.
Processed 10912/13243 matches. 82% complete.
Processed 10973/13243 matches. 82% complete.
Processed 11036/13243 matches. 83% complete.
Processed 11094/13243 matches. 83% complete.
Processed 11156/13243 matches. 84% complete.
Processed 11219/13243 matches. 84% complete.
Processed 11281/13243 matches. 85% complete.
Processed 11342/13243 matches. 85% complete.
Processed 11403/13243 matches. 86% complete.
Processed 11463/13243 matches. 86% complete.
Processed 11524/13243 matches. 87% complete.
Processed 

ConnectionError: HTTPSConnectionPool(host='europe.api.riotgames.com', port=443): Max retries exceeded with url: /lol/match/v5/matches/EUW1_6787515217/timeline (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002C4D489DFD0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
sgamespath='../DataProcessing/TestData/SILVER_GAMES.csv'
jsonfilepath5='D:\RawSilverMatchJsons'
call_api_and_save_response_with_progress(sgamespath, jsonfilepath5)

In [3]:
import csv
import requests
import time
import os

def fetch_and_save_match_data(csv_file_path, output_folder_path):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": "RGAPI-e75b46d3-5e51-4829-b978-68790c3ebf56"
    }

    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    # Count total MatchIDs in the CSV file
    with open(csv_file_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        total_ids = sum(1 for row in reader)

    # Reset file pointer to start
    with open(csv_file_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        current_position = 0
        for row in reader:
            match_id = row[0]
            url = f"https://europe.api.riotgames.com/lol/match/v5/matches/{match_id}"

            response = requests.get(url, headers=headers)
            current_position += 1
            
            if response.status_code == 200:
                output_file_path = os.path.join(output_folder_path, f"{match_id}.json")
                with open(output_file_path, 'w', encoding='utf-8') as outfile:
                    outfile.write(response.text)
            else:
                print(f"Failed to fetch data for MatchID: {match_id}, Status Code: {response.status_code}")

            # Progress update
            if current_position % 250 == 0 or current_position == total_ids:  # Update every 250 requests or on last request
                print(f"Progress: {current_position}/{total_ids}")

            time.sleep(1 / 30)  # Respect the rate limit

    print(f"Finished processing. Total requests: {current_position}.")

In [5]:
csv_file_path="../DataProcessing/TestData/GOLD_GAMES.csv"
output_folder_path="D:\RawGoldPostMatchDataJsons"
fetch_and_save_match_data(csv_file_path, output_folder_path)

Failed to fetch data for MatchID: MatchID, Status Code: 404
Progress: 250/18241
Progress: 500/18241
Progress: 750/18241
Progress: 1000/18241
Progress: 1250/18241
Progress: 1500/18241
Progress: 1750/18241
Progress: 2000/18241
Progress: 2250/18241
Progress: 2500/18241
Failed to fetch data for MatchID: EUW1_6816972157, Status Code: 500
Failed to fetch data for MatchID: EUW1_6814211697, Status Code: 500
Failed to fetch data for MatchID: EUW1_6813995890, Status Code: 500
Failed to fetch data for MatchID: EUW1_6812503391, Status Code: 500
Progress: 2750/18241
Failed to fetch data for MatchID: EUW1_6790397807, Status Code: 503
Progress: 3000/18241
Progress: 3250/18241
Progress: 3500/18241
Progress: 3750/18241
Progress: 4000/18241
Progress: 4250/18241
Progress: 4500/18241
Progress: 4750/18241
Failed to fetch data for MatchID: EUW1_6787799895, Status Code: 503
Failed to fetch data for MatchID: EUW1_6786214171, Status Code: 503
Progress: 5000/18241
Progress: 5250/18241
Progress: 5500/18241
Faile

In [6]:
csv_file_path="../DataProcessing/TestData/SILVER_GAMES.csv"
output_folder_path="D:\RawSilverPostMatchDataJsons"
fetch_and_save_match_data(csv_file_path, output_folder_path)

Failed to fetch data for MatchID: MatchID, Status Code: 404
Progress: 250/17761
Progress: 500/17761
Progress: 750/17761
Progress: 1000/17761
Progress: 1250/17761
Progress: 1500/17761
Progress: 1750/17761
Progress: 2000/17761
Progress: 2250/17761
Progress: 2500/17761
Progress: 2750/17761
Progress: 3000/17761
Progress: 3250/17761
Progress: 3500/17761
Progress: 3750/17761
Progress: 4000/17761
Progress: 4250/17761
Progress: 4500/17761
Progress: 4750/17761
Progress: 5000/17761
Progress: 5250/17761
Progress: 5500/17761
Progress: 5750/17761
Progress: 6000/17761
Progress: 6250/17761
Progress: 6500/17761
Progress: 6750/17761
Progress: 7000/17761
Failed to fetch data for MatchID: EUW1_6803965077, Status Code: 500
Failed to fetch data for MatchID: EUW1_6810178871, Status Code: 500
Failed to fetch data for MatchID: EUW1_6807682507, Status Code: 500
Failed to fetch data for MatchID: EUW1_6805832627, Status Code: 500
Failed to fetch data for MatchID: EUW1_6804601190, Status Code: 500
Failed to fetch

In [7]:
csv_file_path="../DataProcessing/TestData/BRONZE_GAMES.csv"
output_folder_path="D:\RawBronzePostMatchDataJsons"
fetch_and_save_match_data(csv_file_path, output_folder_path)

Failed to fetch data for MatchID: MatchID, Status Code: 404
Progress: 250/13244
Progress: 500/13244
Progress: 750/13244
Progress: 1000/13244
Progress: 1250/13244
Progress: 1500/13244
Progress: 1750/13244
Progress: 2000/13244
Progress: 2250/13244
Progress: 2500/13244
Progress: 2750/13244
Progress: 3000/13244
Progress: 3250/13244
Progress: 3500/13244
Progress: 3750/13244
Progress: 4000/13244
Progress: 4250/13244
Progress: 4500/13244
Progress: 4750/13244
Progress: 5000/13244
Failed to fetch data for MatchID: EUW1_6808099874, Status Code: 500
Failed to fetch data for MatchID: EUW1_6816285025, Status Code: 500
Failed to fetch data for MatchID: EUW1_6816229158, Status Code: 500
Progress: 5250/13244
Progress: 5500/13244
Progress: 5750/13244
Progress: 6000/13244
Progress: 6250/13244
Progress: 6500/13244
Progress: 6750/13244
Progress: 7000/13244
Progress: 7250/13244
Progress: 7500/13244
Progress: 7750/13244
Progress: 8000/13244
Progress: 8250/13244
Progress: 8500/13244
Progress: 8750/13244
Prog

In [8]:
csv_file_path="../DataProcessing/TestData/IRON_GAMES.csv"
output_folder_path="D:\RawIronPostMatchDataJsons"
fetch_and_save_match_data(csv_file_path, output_folder_path)

Failed to fetch data for MatchID: MatchID, Status Code: 404
Progress: 250/16281
Progress: 500/16281
Progress: 750/16281
Progress: 1000/16281
Progress: 1250/16281
Progress: 1500/16281
Progress: 1750/16281
Progress: 2000/16281
Progress: 2250/16281
Progress: 2500/16281
Failed to fetch data for MatchID: EUW1_6788867119, Status Code: 500
Failed to fetch data for MatchID: EUW1_6812456517, Status Code: 500
Failed to fetch data for MatchID: EUW1_6812389379, Status Code: 500
Failed to fetch data for MatchID: EUW1_6773359487, Status Code: 500
Failed to fetch data for MatchID: EUW1_6768953165, Status Code: 500
Failed to fetch data for MatchID: EUW1_6815541276, Status Code: 500
Failed to fetch data for MatchID: EUW1_6804319447, Status Code: 500
Failed to fetch data for MatchID: EUW1_6799605922, Status Code: 500
Progress: 2750/16281
Progress: 3000/16281
Progress: 3250/16281
Progress: 3500/16281
Progress: 3750/16281
Progress: 4000/16281
Progress: 4250/16281
Progress: 4500/16281
Progress: 4750/16281


In [9]:
csv_file_path="../DataProcessing/TestData/PLATINUM_GAMES.csv"
output_folder_path="D:\RawPlatinumPostMatchDataJsons"
fetch_and_save_match_data(csv_file_path, output_folder_path)

Failed to fetch data for MatchID: MatchID, Status Code: 404
Progress: 250/20281
Progress: 500/20281
Progress: 750/20281
Progress: 1000/20281
Progress: 1250/20281
Failed to fetch data for MatchID: EUW1_6790432560, Status Code: 500
Failed to fetch data for MatchID: EUW1_6790369664, Status Code: 500
Failed to fetch data for MatchID: EUW1_6790339288, Status Code: 500
Failed to fetch data for MatchID: EUW1_6790293028, Status Code: 500
Failed to fetch data for MatchID: EUW1_6778932575, Status Code: 500
Failed to fetch data for MatchID: EUW1_6778843383, Status Code: 500
Failed to fetch data for MatchID: EUW1_6814185144, Status Code: 500
Failed to fetch data for MatchID: EUW1_6812227840, Status Code: 500
Failed to fetch data for MatchID: EUW1_6809749100, Status Code: 500
Progress: 1500/20281
Progress: 1750/20281
Progress: 2000/20281
Progress: 2250/20281
Progress: 2500/20281
Progress: 2750/20281
Failed to fetch data for MatchID: EUW1_6775347083, Status Code: 404
Progress: 3000/20281
Progress: 3

In [10]:
csv_file_path="../DataProcessing/TestData/EMERALD_GAMES.csv"
output_folder_path="D:\RawEmeraldPostMatchDataJsons"
fetch_and_save_match_data(csv_file_path, output_folder_path)

Failed to fetch data for MatchID: MatchID, Status Code: 404
Progress: 250/22001
Progress: 500/22001
Progress: 750/22001
Progress: 1000/22001
Progress: 1250/22001
Progress: 1500/22001
Progress: 1750/22001
Progress: 2000/22001
Progress: 2250/22001
Progress: 2500/22001
Progress: 2750/22001
Progress: 3000/22001
Progress: 3250/22001
Progress: 3500/22001
Progress: 3750/22001
Progress: 4000/22001
Progress: 4250/22001
Progress: 4500/22001
Progress: 4750/22001
Progress: 5000/22001
Progress: 5250/22001
Progress: 5500/22001
Progress: 5750/22001
Progress: 6000/22001
Progress: 6250/22001
Progress: 6500/22001
Progress: 6750/22001
Progress: 7000/22001
Progress: 7250/22001
Progress: 7500/22001
Progress: 7750/22001
Progress: 8000/22001
Progress: 8250/22001
Progress: 8500/22001
Progress: 8750/22001
Progress: 9000/22001
Progress: 9250/22001
Progress: 9500/22001
Progress: 9750/22001
Progress: 10000/22001
Progress: 10250/22001
Progress: 10500/22001
Progress: 10750/22001
Progress: 11000/22001
Progress: 1125

In [11]:
csv_file_path="../DataProcessing/TestData/DIAMOND_GAMES.csv"
output_folder_path="D:\RawDiamondPostMatchDataJsons"
fetch_and_save_match_data(csv_file_path, output_folder_path)

Failed to fetch data for MatchID: MatchID, Status Code: 404
Progress: 250/27981
Progress: 500/27981
Progress: 750/27981
Progress: 1000/27981
Progress: 1250/27981
Progress: 1500/27981
Progress: 1750/27981
Progress: 2000/27981
Progress: 2250/27981
Progress: 2500/27981
Progress: 2750/27981
Progress: 3000/27981
Progress: 3250/27981
Failed to fetch data for MatchID: EUW1_6795887958, Status Code: 500
Failed to fetch data for MatchID: EUW1_6790178007, Status Code: 500
Failed to fetch data for MatchID: EUW1_6788518574, Status Code: 500
Failed to fetch data for MatchID: EUW1_6814869143, Status Code: 500
Failed to fetch data for MatchID: EUW1_6803443917, Status Code: 500
Failed to fetch data for MatchID: EUW1_6816113646, Status Code: 500
Progress: 3500/27981
Progress: 3750/27981
Progress: 4000/27981
Progress: 4250/27981
Progress: 4500/27981
Progress: 4750/27981
Failed to fetch data for MatchID: EUW1_6809487215, Status Code: 500
Failed to fetch data for MatchID: EUW1_6807759435, Status Code: 500


In [ ]:
import csv
import requests
import time
import os

def fetch_and_save_match_data(csv_file_path, output_folder_path):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": "RGAPI-e75b46d3-5e51-4829-b978-68790c3ebf56"
    }

    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    # Count total MatchIDs in the CSV file
    with open(csv_file_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        total_ids = sum(1 for row in reader)

    # Reset file pointer to start
    with open(csv_file_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        current_position = 0
        for row in reader:
            match_id = row[0]
            url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}"

            response = requests.get(url, headers=headers)
            current_position += 1
            
            if response.status_code == 200:
                output_file_path = os.path.join(output_folder_path, f"{match_id}.json")
                with open(output_file_path, 'w', encoding='utf-8') as outfile:
                    outfile.write(response.text)
            else:
                print(f"Failed to fetch data for MatchID: {match_id}, Status Code: {response.status_code}")

            # Progress update
            if current_position % 250 == 0 or current_position == total_ids:  # Update every 250 requests or on last request
                print(f"Progress: {current_position}/{total_ids}")

            time.sleep(1 / 30)  # Respect the rate limit

    print(f"Finished processing. Total requests: {current_position}.")

In [ ]:
csv_file_path="../DataProcessing/TestData/GOLD_GAMES_KOREA.csv"
output_folder_path="D:\RawGoldPostMatchDataJsonsKorea"
fetch_and_save_match_data(csv_file_path, output_folder_path)

In [ ]:
csv_file_path="../DataProcessing/TestData/SILVER_GAMES_KOREA.csv"
output_folder_path="D:\RawSilverPostMatchDataJsonsKorea"
fetch_and_save_match_data(csv_file_path, output_folder_path)

In [ ]:
csv_file_path="../DataProcessing/TestData/BRONZE_GAMES_KOREA.csv"
output_folder_path="D:\RawBronzePostMatchDataJsonsKorea"
fetch_and_save_match_data(csv_file_path, output_folder_path)

In [ ]:
csv_file_path="../DataProcessing/TestData/IRON_GAMES_KOREA.csv"
output_folder_path="D:\RawIronPostMatchDataJsonsKorea"
fetch_and_save_match_data(csv_file_path, output_folder_path)

In [ ]:
csv_file_path="../DataProcessing/TestData/PLATINUM_GAMES_KOREA.csv"
output_folder_path="D:\RawPlatinumPostMatchDataJsonsKorea"
fetch_and_save_match_data(csv_file_path, output_folder_path)

In [ ]:
csv_file_path="../DataProcessing/TestData/EMERALD_GAMES_KOREA.csv"
output_folder_path="D:\RawEmeraldPostMatchDataJsonsKorea"
fetch_and_save_match_data(csv_file_path, output_folder_path)

In [ ]:
csv_file_path="../DataProcessing/TestData/DIAMOND_GAMES_KOREA.csv"
output_folder_path="D:\RawDiamondPostMatchDataJsonsKorea"
fetch_and_save_match_data(csv_file_path, output_folder_path)

In [12]:
import os

def find_exclusive_files(folder1, folder2):
    # Get the list of file names in both folders
    files_folder1 = set(os.listdir(folder1))
    files_folder2 = set(os.listdir(folder2))
    
    # Find files that are exclusive to each folder (i.e., present in one folder but not the other)
    exclusive_to_folder1 = files_folder1 - files_folder2
    exclusive_to_folder2 = files_folder2 - files_folder1
    
    # Calculate the total amount of exclusive files
    total_exclusive_folder1 = len(exclusive_to_folder1)
    total_exclusive_folder2 = len(exclusive_to_folder2)
    
    # Print out the exclusive files and their total count
    if exclusive_to_folder1:
        print(f"Files exclusive to {folder1}:")
        for file in exclusive_to_folder1:
            print(file)
        print(f"Total exclusive files in {folder1}: {total_exclusive_folder1}")
    else:
        print(f"No files exclusive to {folder1}")
        
    if exclusive_to_folder2:
        print(f"Files exclusive to {folder2}:")
        for file in exclusive_to_folder2:
            print(file)
        print(f"Total exclusive files in {folder2}: {total_exclusive_folder2}")
    else:
        print(f"No files exclusive to {folder2}")

In [16]:
folder1="D:\EUW\CleanedBronzeMatchJsons"
folder2="D:\EUW\CleanedBronzePostMatchDataJsons"
find_exclusive_files(folder1, folder2)

Files exclusive to D:\EUW\CleanedBronzeMatchJsons:
EUW1_6777707241.json
EUW1_6811247239.json
EUW1_6797152521.json
EUW1_6797937871.json
EUW1_6792713776.json
EUW1_6815601717.json
EUW1_6794256831.json
EUW1_6798158877.json
EUW1_6798398805.json
EUW1_6781780096.json
EUW1_6800515606.json
EUW1_6779163920.json
EUW1_6813298317.json
EUW1_6790991844.json
EUW1_6791065072.json
EUW1_6806361543.json
EUW1_6802042239.json
EUW1_6812040435.json
EUW1_6793646812.json
EUW1_6792066678.json
EUW1_6802514830.json
EUW1_6813347532.json
EUW1_6798463534.json
EUW1_6769459475.json
EUW1_6768075570.json
EUW1_6806046350.json
EUW1_6797310912.json
EUW1_6793519545.json
EUW1_6812641124.json
EUW1_6803627400.json
EUW1_6814810141.json
EUW1_6785105476.json
EUW1_6793660899.json
EUW1_6790645167.json
EUW1_6798783181.json
EUW1_6786878207.json
EUW1_6806220175.json
EUW1_6816458695.json
EUW1_6811571214.json
EUW1_6802419146.json
EUW1_6814102495.json
EUW1_6802800676.json
EUW1_6812114372.json
EUW1_6814972425.json
EUW1_6810813128.json
EUW1

In [17]:
folder1="D:\EUW\CleanedIronMatchJsons"
folder2="D:\EUW\CleanedIronPostMatchDataJsons"
find_exclusive_files(folder1, folder2)

Files exclusive to D:\EUW\CleanedIronMatchJsons:
EUW1_6798822599.json
EUW1_6815268335.json
EUW1_6797063791.json
EUW1_6779955907.json
EUW1_6806073215.json
EUW1_6786844642.json
EUW1_6813416960.json
EUW1_6798762028.json
EUW1_6795685277.json
EUW1_6773359487.json
EUW1_6771852274.json
EUW1_6792429789.json
EUW1_6810913446.json
EUW1_6812155790.json
EUW1_6812040599.json
EUW1_6812456517.json
EUW1_6804341218.json
EUW1_6812389379.json
EUW1_6788867119.json
EUW1_6798599010.json
EUW1_6768953165.json
EUW1_6814379960.json
EUW1_6815541276.json
EUW1_6799605922.json
EUW1_6813591397.json
EUW1_6816005054.json
EUW1_6810450477.json
Total exclusive files in D:\EUW\CleanedIronMatchJsons: 27
Files exclusive to D:\EUW\CleanedIronPostMatchDataJsons:
EUW1_6784692455.json
EUW1_6816112661.json
EUW1_6815504869.json
EUW1_6816126357.json
EUW1_6792899846.json
EUW1_6814420506.json
EUW1_6785809960.json
EUW1_6803681283.json
EUW1_6807046653.json
EUW1_6804505410.json
EUW1_6800687372.json
EUW1_6784589375.json
EUW1_6816339376.j

In [18]:
folder1="D:\EUW\CleanedSilverMatchJsons"
folder2="D:\EUW\CleanedSilverPostMatchDataJsons"
find_exclusive_files(folder1, folder2)

Files exclusive to D:\EUW\CleanedSilverMatchJsons:
EUW1_6810178871.json
EUW1_6798553352.json
EUW1_6789500086.json
EUW1_6803965077.json
EUW1_6799621951.json
EUW1_6812209559.json
EUW1_6807527379.json
EUW1_6816487714.json
EUW1_6815141216.json
EUW1_6805245958.json
EUW1_6805832627.json
EUW1_6804601190.json
EUW1_6791654504.json
EUW1_6807682507.json
EUW1_6802094366.json
Total exclusive files in D:\EUW\CleanedSilverMatchJsons: 15
Files exclusive to D:\EUW\CleanedSilverPostMatchDataJsons:
EUW1_6813318224.json
EUW1_6810246703.json
EUW1_6810188250.json
EUW1_6810286036.json
EUW1_6794476785.json
EUW1_6773474443.json
EUW1_6788248297.json
EUW1_6794762521.json
EUW1_6805133256.json
EUW1_6795777912.json
EUW1_6816248247.json
EUW1_6809136857.json
EUW1_6771902575.json
EUW1_6810294399.json
EUW1_6811419399.json
EUW1_6816385211.json
EUW1_6812216715.json
EUW1_6806167789.json
EUW1_6794727737.json
EUW1_6810174546.json
EUW1_6794683451.json
EUW1_6793322868.json
EUW1_6775072842.json
EUW1_6810362187.json
EUW1_677625

In [19]:
folder1="D:\EUW\CleanedGoldMatchJsons"
folder2="D:\EUW\CleanedGoldPostMatchDataJsons"
find_exclusive_files(folder1, folder2)

Files exclusive to D:\EUW\CleanedGoldMatchJsons:
EUW1_6814211697.json
EUW1_6800932738.json
EUW1_6813995890.json
EUW1_6812503391.json
EUW1_6814874203.json
EUW1_6786214171.json
EUW1_6787799895.json
EUW1_6790397807.json
EUW1_6797694595.json
EUW1_6816972157.json
Total exclusive files in D:\EUW\CleanedGoldMatchJsons: 10
Files exclusive to D:\EUW\CleanedGoldPostMatchDataJsons:
EUW1_6812318901.json
EUW1_6802583076.json
EUW1_6815936439.json
EUW1_6799645422.json
EUW1_6782647562.json
EUW1_6801617928.json
EUW1_6808249274.json
EUW1_6769111988.json
EUW1_6774220639.json
EUW1_6810543827.json
EUW1_6810478427.json
EUW1_6817119264.json
EUW1_6803050474.json
EUW1_6806000027.json
EUW1_6774144258.json
EUW1_6780392475.json
EUW1_6801494379.json
EUW1_6810332743.json
Total exclusive files in D:\EUW\CleanedGoldPostMatchDataJsons: 18


In [20]:
folder1="D:\EUW\CleanedDiamondMatchJsons"
folder2="D:\EUW\CleanedDiamondPostMatchDataJsons"
find_exclusive_files(folder1, folder2)

Files exclusive to D:\EUW\CleanedDiamondMatchJsons:
EUW1_6795887958.json
EUW1_6793378218.json
EUW1_6782024207.json
EUW1_6807759435.json
EUW1_6815893410.json
EUW1_6799694965.json
EUW1_6790178007.json
EUW1_6788518574.json
EUW1_6780550567.json
EUW1_6803443917.json
EUW1_6799568713.json
EUW1_6815906665.json
EUW1_6799537800.json
EUW1_6809487215.json
EUW1_6811238553.json
EUW1_6782169901.json
EUW1_6807647731.json
EUW1_6801845125.json
EUW1_6813066652.json
Total exclusive files in D:\EUW\CleanedDiamondMatchJsons: 19
Files exclusive to D:\EUW\CleanedDiamondPostMatchDataJsons:
EUW1_6814088190.json
EUW1_6807055504.json
EUW1_6781838868.json
EUW1_6798912353.json
EUW1_6805640663.json
EUW1_6800403127.json
EUW1_6778569907.json
EUW1_6779702603.json
EUW1_6797485293.json
EUW1_6804311387.json
EUW1_6817172452.json
EUW1_6809428604.json
EUW1_6808531701.json
EUW1_6806972142.json
EUW1_6812288446.json
EUW1_6811182242.json
EUW1_6801957864.json
EUW1_6810929580.json
EUW1_6782403596.json
EUW1_6805000899.json
EUW1_680

In [21]:
folder1="D:\EUW\CleanedEmeraldMatchJsons"
folder2="D:\EUW\CleanedEmeraldPostMatchDataJsons"
find_exclusive_files(folder1, folder2)

No files exclusive to D:\EUW\CleanedEmeraldMatchJsons
Files exclusive to D:\EUW\CleanedEmeraldPostMatchDataJsons:
EUW1_6810633717.json
EUW1_6814218866.json
EUW1_6807743577.json
EUW1_6789037911.json
EUW1_6808286727.json
EUW1_6798727666.json
EUW1_6807319047.json
EUW1_6792696774.json
EUW1_6808572122.json
EUW1_6804012055.json
EUW1_6792412189.json
EUW1_6793640551.json
EUW1_6805718200.json
EUW1_6801358672.json
EUW1_6802044308.json
EUW1_6814416743.json
EUW1_6815418494.json
EUW1_6809613854.json
EUW1_6798184996.json
EUW1_6803378247.json
EUW1_6792089595.json
EUW1_6803845321.json
EUW1_6802740208.json
EUW1_6789113859.json
EUW1_6817452372.json
EUW1_6810420323.json
EUW1_6817471244.json
EUW1_6798471047.json
EUW1_6809304401.json
EUW1_6812142629.json
EUW1_6802867063.json
Total exclusive files in D:\EUW\CleanedEmeraldPostMatchDataJsons: 31


In [22]:
folder1="D:\EUW\CleanedPlatinumMatchJsons"
folder2="D:\EUW\CleanedPlatinumPostMatchDataJsons"
find_exclusive_files(folder1, folder2)

Files exclusive to D:\EUW\CleanedPlatinumMatchJsons:
EUW1_6791252063.json
EUW1_6808966258.json
EUW1_6801459698.json
EUW1_6786576795.json
EUW1_6810462789.json
EUW1_6790339288.json
EUW1_6817204370.json
EUW1_6815532148.json
EUW1_6790432560.json
EUW1_6793435242.json
EUW1_6807007202.json
EUW1_6778843383.json
EUW1_6806830623.json
EUW1_6796801839.json
EUW1_6799613484.json
EUW1_6794757840.json
EUW1_6806927580.json
EUW1_6806370376.json
EUW1_6790293028.json
EUW1_6817047813.json
EUW1_6814185144.json
EUW1_6812227840.json
EUW1_6798534624.json
EUW1_6791137782.json
EUW1_6793935968.json
EUW1_6798631319.json
EUW1_6795486709.json
EUW1_6807593044.json
EUW1_6816384169.json
EUW1_6809749100.json
EUW1_6790369664.json
EUW1_6806858462.json
EUW1_6778932575.json
Total exclusive files in D:\EUW\CleanedPlatinumMatchJsons: 33
Files exclusive to D:\EUW\CleanedPlatinumPostMatchDataJsons:
EUW1_6807584802.json
EUW1_6786707612.json
EUW1_6779244430.json
EUW1_6805838768.json
EUW1_6806962327.json
EUW1_6810465431.json
EUW1_

In [23]:
import os

def find_and_delete_exclusive_files(folder1, folder2):
    # Get the list of file names in both folders
    files_folder1 = set(os.listdir(folder1))
    files_folder2 = set(os.listdir(folder2))
    
    # Find files that are exclusive to each folder (i.e., present in one folder but not the other)
    exclusive_to_folder1 = files_folder1 - files_folder2
    exclusive_to_folder2 = files_folder2 - files_folder1
    
    # Calculate the total amount of exclusive files
    total_exclusive_folder1 = len(exclusive_to_folder1)
    total_exclusive_folder2 = len(exclusive_to_folder2)
    
    # Print out the exclusive files and their total count
    if exclusive_to_folder1:
        print(f"Deleting files exclusive to {folder1}:")
        for file in exclusive_to_folder1:
            file_path = os.path.join(folder1, file)
            os.remove(file_path)
            print(f"Deleted {file}")
        print(f"Total exclusive files deleted in {folder1}: {total_exclusive_folder1}")
    else:
        print(f"No files exclusive to {folder1}")
        
    if exclusive_to_folder2:
        print(f"Deleting files exclusive to {folder2}:")
        for file in exclusive_to_folder2:
            file_path = os.path.join(folder2, file)
            os.remove(file_path)
            print(f"Deleted {file}")
        print(f"Total exclusive files deleted in {folder2}: {total_exclusive_folder2}")
    else:
        print(f"No files exclusive to {folder2}")

In [24]:
folder1="D:\EUW\CleanedBronzeMatchJsons"
folder2="D:\EUW\CleanedBronzePostMatchDataJsons"
find_and_delete_exclusive_files(folder1, folder2)

Deleting files exclusive to D:\EUW\CleanedBronzeMatchJsons:
Deleted EUW1_6777707241.json
Deleted EUW1_6811247239.json
Deleted EUW1_6797152521.json
Deleted EUW1_6797937871.json
Deleted EUW1_6792713776.json
Deleted EUW1_6815601717.json
Deleted EUW1_6794256831.json
Deleted EUW1_6798158877.json
Deleted EUW1_6798398805.json
Deleted EUW1_6781780096.json
Deleted EUW1_6800515606.json
Deleted EUW1_6779163920.json
Deleted EUW1_6813298317.json
Deleted EUW1_6790991844.json
Deleted EUW1_6791065072.json
Deleted EUW1_6806361543.json
Deleted EUW1_6802042239.json
Deleted EUW1_6812040435.json
Deleted EUW1_6793646812.json
Deleted EUW1_6792066678.json
Deleted EUW1_6802514830.json
Deleted EUW1_6813347532.json
Deleted EUW1_6798463534.json
Deleted EUW1_6769459475.json
Deleted EUW1_6768075570.json
Deleted EUW1_6806046350.json
Deleted EUW1_6797310912.json
Deleted EUW1_6793519545.json
Deleted EUW1_6812641124.json
Deleted EUW1_6803627400.json
Deleted EUW1_6814810141.json
Deleted EUW1_6785105476.json
Deleted EUW1

Deleted EUW1_6795106108.json
Deleted EUW1_6786467053.json
Deleted EUW1_6781647932.json
Deleted EUW1_6814702308.json
Deleted EUW1_6816633219.json
Deleted EUW1_6805982499.json
Deleted EUW1_6802066671.json
Deleted EUW1_6814510202.json
Deleted EUW1_6789763469.json
Deleted EUW1_6800591069.json
Deleted EUW1_6784978605.json
Deleted EUW1_6769155723.json
Deleted EUW1_6792238669.json
Deleted EUW1_6813162302.json
Deleted EUW1_6764570291.json
Deleted EUW1_6796637634.json
Deleted EUW1_6799281752.json
Deleted EUW1_6810286297.json
Deleted EUW1_6780494038.json
Deleted EUW1_6814832516.json
Deleted EUW1_6815632188.json
Deleted EUW1_6782465343.json
Deleted EUW1_6797357727.json
Deleted EUW1_6783072240.json
Deleted EUW1_6784651830.json
Deleted EUW1_6815847748.json
Deleted EUW1_6812537133.json
Deleted EUW1_6812327884.json
Deleted EUW1_6795955562.json
Deleted EUW1_6797519859.json
Deleted EUW1_6783713649.json
Deleted EUW1_6789787638.json
Deleted EUW1_6793528546.json
Deleted EUW1_6783373388.json
Deleted EUW1_6

Deleted EUW1_6816542992.json
Deleted EUW1_6811404566.json
Deleted EUW1_6792471134.json
Deleted EUW1_6781024373.json
Deleted EUW1_6782828360.json
Deleted EUW1_6799896848.json
Deleted EUW1_6815065183.json
Deleted EUW1_6803974709.json
Deleted EUW1_6794563117.json
Deleted EUW1_6812085856.json
Deleted EUW1_6803108392.json
Deleted EUW1_6813407125.json
Deleted EUW1_6802490249.json
Deleted EUW1_6801890033.json
Deleted EUW1_6815058874.json
Deleted EUW1_6805567614.json
Deleted EUW1_6781049644.json
Deleted EUW1_6799690421.json
Deleted EUW1_6808521759.json
Deleted EUW1_6808597316.json
Deleted EUW1_6795411741.json
Deleted EUW1_6812091680.json
Deleted EUW1_6813338451.json
Deleted EUW1_6806995650.json
Deleted EUW1_6805374354.json
Deleted EUW1_6770426899.json
Deleted EUW1_6791212738.json
Deleted EUW1_6781579344.json
Deleted EUW1_6775736377.json
Deleted EUW1_6814632712.json
Deleted EUW1_6815341410.json
Deleted EUW1_6812736822.json
Deleted EUW1_6813321858.json
Deleted EUW1_6776194067.json
Deleted EUW1_6

Deleted EUW1_6789300330.json
Deleted EUW1_6805137912.json
Deleted EUW1_6795204724.json
Deleted EUW1_6800730954.json
Deleted EUW1_6802804466.json
Deleted EUW1_6790742082.json
Deleted EUW1_6791291844.json
Deleted EUW1_6770445014.json
Deleted EUW1_6788454764.json
Deleted EUW1_6803937890.json
Deleted EUW1_6807478871.json
Deleted EUW1_6791519614.json
Deleted EUW1_6775787671.json
Deleted EUW1_6815103294.json
Deleted EUW1_6805483802.json
Deleted EUW1_6789415460.json
Deleted EUW1_6782971456.json
Deleted EUW1_6807849947.json
Deleted EUW1_6794962711.json
Deleted EUW1_6815341087.json
Deleted EUW1_6799875436.json
Deleted EUW1_6810402514.json
Deleted EUW1_6809467677.json
Deleted EUW1_6789284950.json
Deleted EUW1_6806158736.json
Deleted EUW1_6810230751.json
Deleted EUW1_6799029320.json
Deleted EUW1_6793687573.json
Deleted EUW1_6816106329.json
Deleted EUW1_6798075731.json
Deleted EUW1_6797123521.json
Deleted EUW1_6779225852.json
Deleted EUW1_6786034432.json
Deleted EUW1_6778532527.json
Deleted EUW1_6

Deleted EUW1_6793595534.json
Deleted EUW1_6778904168.json
Deleted EUW1_6762454322.json
Deleted EUW1_6782772973.json
Deleted EUW1_6807640380.json
Deleted EUW1_6808223980.json
Deleted EUW1_6816421690.json
Deleted EUW1_6795160769.json
Deleted EUW1_6781445248.json
Deleted EUW1_6814368843.json
Deleted EUW1_6780956904.json
Deleted EUW1_6801228140.json
Deleted EUW1_6808705789.json
Deleted EUW1_6816657356.json
Deleted EUW1_6797031919.json
Deleted EUW1_6808103746.json
Deleted EUW1_6815753946.json
Deleted EUW1_6812319746.json
Deleted EUW1_6791561177.json
Deleted EUW1_6796100851.json
Deleted EUW1_6784139646.json
Deleted EUW1_6789536999.json
Deleted EUW1_6799627867.json
Deleted EUW1_6794499065.json
Deleted EUW1_6765628409.json
Deleted EUW1_6808062273.json
Deleted EUW1_6778598691.json
Deleted EUW1_6791378231.json
Deleted EUW1_6801868543.json
Deleted EUW1_6790445084.json
Deleted EUW1_6813186302.json
Deleted EUW1_6787341542.json
Deleted EUW1_6799503226.json
Deleted EUW1_6810269933.json
Deleted EUW1_6

Deleted EUW1_6793890060.json
Deleted EUW1_6806829869.json
Deleted EUW1_6815719614.json
Deleted EUW1_6816630239.json
Deleted EUW1_6798404599.json
Deleted EUW1_6790691226.json
Deleted EUW1_6783842560.json
Deleted EUW1_6795701652.json
Deleted EUW1_6813541599.json
Deleted EUW1_6814379166.json
Deleted EUW1_6793784996.json
Deleted EUW1_6790555925.json
Deleted EUW1_6811997742.json
Deleted EUW1_6805235511.json
Deleted EUW1_6812578574.json
Deleted EUW1_6796279975.json
Deleted EUW1_6803693942.json
Deleted EUW1_6766211317.json
Deleted EUW1_6785482069.json
Deleted EUW1_6779014087.json
Deleted EUW1_6810570825.json
Deleted EUW1_6793013653.json
Deleted EUW1_6814732747.json
Deleted EUW1_6812661014.json
Deleted EUW1_6803278199.json
Deleted EUW1_6806773603.json
Deleted EUW1_6811226747.json
Deleted EUW1_6808720873.json
Deleted EUW1_6815878775.json
Deleted EUW1_6802390631.json
Deleted EUW1_6788623643.json
Deleted EUW1_6777386146.json
Deleted EUW1_6805802488.json
Deleted EUW1_6799187012.json
Deleted EUW1_6

Deleted EUW1_6789742836.json
Deleted EUW1_6804422157.json
Deleted EUW1_6779616364.json
Deleted EUW1_6803730988.json
Deleted EUW1_6773549485.json
Deleted EUW1_6791330285.json
Deleted EUW1_6816505787.json
Deleted EUW1_6813782833.json
Deleted EUW1_6815355386.json
Deleted EUW1_6813033607.json
Deleted EUW1_6815017830.json
Deleted EUW1_6774086444.json
Deleted EUW1_6803881118.json
Deleted EUW1_6797024447.json
Deleted EUW1_6803055904.json
Deleted EUW1_6783840491.json
Deleted EUW1_6803544666.json
Deleted EUW1_6793961622.json
Deleted EUW1_6814559595.json
Deleted EUW1_6800529135.json
Deleted EUW1_6781159827.json
Deleted EUW1_6791077424.json
Deleted EUW1_6772006815.json
Deleted EUW1_6815177339.json
Deleted EUW1_6814728704.json
Deleted EUW1_6797782460.json
Deleted EUW1_6798538721.json
Deleted EUW1_6794793171.json
Deleted EUW1_6810531224.json
Deleted EUW1_6792612403.json
Deleted EUW1_6796349859.json
Deleted EUW1_6792069883.json
Deleted EUW1_6801716689.json
Deleted EUW1_6796416648.json
Deleted EUW1_6

Deleted EUW1_6809380101.json
Deleted EUW1_6816503124.json
Deleted EUW1_6811035387.json
Deleted EUW1_6789092780.json
Deleted EUW1_6811171962.json
Deleted EUW1_6815650437.json
Deleted EUW1_6798496857.json
Deleted EUW1_6776384684.json
Deleted EUW1_6809140109.json
Deleted EUW1_6775932844.json
Deleted EUW1_6802328818.json
Deleted EUW1_6801170976.json
Deleted EUW1_6805777471.json
Deleted EUW1_6793186370.json
Deleted EUW1_6808524990.json
Deleted EUW1_6805306682.json
Deleted EUW1_6796304073.json
Deleted EUW1_6806461850.json
Deleted EUW1_6812939112.json
Deleted EUW1_6782922918.json
Deleted EUW1_6791660859.json
Deleted EUW1_6805192443.json
Deleted EUW1_6772588441.json
Deleted EUW1_6812793565.json
Deleted EUW1_6810449747.json
Deleted EUW1_6779564955.json
Deleted EUW1_6816613962.json
Deleted EUW1_6807022561.json
Deleted EUW1_6796871766.json
Deleted EUW1_6810831718.json
Deleted EUW1_6809433052.json
Deleted EUW1_6791953439.json
Deleted EUW1_6803188802.json
Deleted EUW1_6800745358.json
Deleted EUW1_6

Deleted EUW1_6808488876.json
Deleted EUW1_6809270254.json
Deleted EUW1_6799605175.json
Deleted EUW1_6797245586.json
Deleted EUW1_6813025747.json
Deleted EUW1_6815797667.json
Deleted EUW1_6809827813.json
Deleted EUW1_6780599996.json
Deleted EUW1_6780748717.json
Deleted EUW1_6784978853.json
Deleted EUW1_6795269821.json
Deleted EUW1_6792946795.json
Deleted EUW1_6814352879.json
Deleted EUW1_6811627405.json
Deleted EUW1_6783710935.json
Deleted EUW1_6780292680.json
Deleted EUW1_6799045427.json
Deleted EUW1_6790059234.json
Deleted EUW1_6813837441.json
Deleted EUW1_6797997407.json
Deleted EUW1_6813263650.json
Deleted EUW1_6813603318.json
Deleted EUW1_6816646166.json
Deleted EUW1_6814774302.json
Deleted EUW1_6809004768.json
Deleted EUW1_6814402204.json
Deleted EUW1_6788164628.json
Deleted EUW1_6809597365.json
Deleted EUW1_6782305335.json
Deleted EUW1_6815986678.json
Deleted EUW1_6775367057.json
Deleted EUW1_6772765581.json
Deleted EUW1_6791042673.json
Deleted EUW1_6805721574.json
Deleted EUW1_6

In [25]:
folder1="D:\EUW\CleanedBronzeMatchJsons"
folder2="D:\EUW\CleanedBronzePostMatchDataJsons"
find_exclusive_files(folder1, folder2)

No files exclusive to D:\EUW\CleanedBronzeMatchJsons
No files exclusive to D:\EUW\CleanedBronzePostMatchDataJsons


In [26]:
folder1="D:\EUW\CleanedIronMatchJsons"
folder2="D:\EUW\CleanedIronPostMatchDataJsons"
find_and_delete_exclusive_files(folder1, folder2)

Deleting files exclusive to D:\EUW\CleanedIronMatchJsons:
Deleted EUW1_6798822599.json
Deleted EUW1_6815268335.json
Deleted EUW1_6797063791.json
Deleted EUW1_6779955907.json
Deleted EUW1_6806073215.json
Deleted EUW1_6786844642.json
Deleted EUW1_6813416960.json
Deleted EUW1_6798762028.json
Deleted EUW1_6795685277.json
Deleted EUW1_6773359487.json
Deleted EUW1_6771852274.json
Deleted EUW1_6792429789.json
Deleted EUW1_6810913446.json
Deleted EUW1_6812155790.json
Deleted EUW1_6812040599.json
Deleted EUW1_6812456517.json
Deleted EUW1_6804341218.json
Deleted EUW1_6812389379.json
Deleted EUW1_6788867119.json
Deleted EUW1_6798599010.json
Deleted EUW1_6768953165.json
Deleted EUW1_6814379960.json
Deleted EUW1_6815541276.json
Deleted EUW1_6799605922.json
Deleted EUW1_6813591397.json
Deleted EUW1_6816005054.json
Deleted EUW1_6810450477.json
Total exclusive files deleted in D:\EUW\CleanedIronMatchJsons: 27
Deleting files exclusive to D:\EUW\CleanedIronPostMatchDataJsons:
Deleted EUW1_6784692455.jso

In [27]:
folder1="D:\EUW\CleanedIronMatchJsons"
folder2="D:\EUW\CleanedIronPostMatchDataJsons"
find_exclusive_files(folder1, folder2)

No files exclusive to D:\EUW\CleanedIronMatchJsons
No files exclusive to D:\EUW\CleanedIronPostMatchDataJsons


In [28]:
folder1="D:\EUW\CleanedSilverMatchJsons"
folder2="D:\EUW\CleanedSilverPostMatchDataJsons"
find_and_delete_exclusive_files(folder1, folder2)

Deleting files exclusive to D:\EUW\CleanedSilverMatchJsons:
Deleted EUW1_6810178871.json
Deleted EUW1_6798553352.json
Deleted EUW1_6789500086.json
Deleted EUW1_6803965077.json
Deleted EUW1_6799621951.json
Deleted EUW1_6812209559.json
Deleted EUW1_6807527379.json
Deleted EUW1_6816487714.json
Deleted EUW1_6815141216.json
Deleted EUW1_6805245958.json
Deleted EUW1_6805832627.json
Deleted EUW1_6804601190.json
Deleted EUW1_6791654504.json
Deleted EUW1_6807682507.json
Deleted EUW1_6802094366.json
Total exclusive files deleted in D:\EUW\CleanedSilverMatchJsons: 15
Deleting files exclusive to D:\EUW\CleanedSilverPostMatchDataJsons:
Deleted EUW1_6813318224.json
Deleted EUW1_6810246703.json
Deleted EUW1_6810188250.json
Deleted EUW1_6810286036.json
Deleted EUW1_6794476785.json
Deleted EUW1_6773474443.json
Deleted EUW1_6788248297.json
Deleted EUW1_6794762521.json
Deleted EUW1_6805133256.json
Deleted EUW1_6795777912.json
Deleted EUW1_6816248247.json
Deleted EUW1_6809136857.json
Deleted EUW1_67719025

In [29]:
folder1="D:\EUW\CleanedSilverMatchJsons"
folder2="D:\EUW\CleanedSilverPostMatchDataJsons"
find_exclusive_files(folder1, folder2)

No files exclusive to D:\EUW\CleanedSilverMatchJsons
No files exclusive to D:\EUW\CleanedSilverPostMatchDataJsons


In [30]:
folder1="D:\EUW\CleanedGoldMatchJsons"
folder2="D:\EUW\CleanedGoldPostMatchDataJsons"
find_and_delete_exclusive_files(folder1, folder2)

Deleting files exclusive to D:\EUW\CleanedGoldMatchJsons:
Deleted EUW1_6814211697.json
Deleted EUW1_6800932738.json
Deleted EUW1_6813995890.json
Deleted EUW1_6812503391.json
Deleted EUW1_6814874203.json
Deleted EUW1_6786214171.json
Deleted EUW1_6787799895.json
Deleted EUW1_6790397807.json
Deleted EUW1_6797694595.json
Deleted EUW1_6816972157.json
Total exclusive files deleted in D:\EUW\CleanedGoldMatchJsons: 10
Deleting files exclusive to D:\EUW\CleanedGoldPostMatchDataJsons:
Deleted EUW1_6812318901.json
Deleted EUW1_6802583076.json
Deleted EUW1_6815936439.json
Deleted EUW1_6799645422.json
Deleted EUW1_6782647562.json
Deleted EUW1_6801617928.json
Deleted EUW1_6808249274.json
Deleted EUW1_6769111988.json
Deleted EUW1_6774220639.json
Deleted EUW1_6810543827.json
Deleted EUW1_6810478427.json
Deleted EUW1_6817119264.json
Deleted EUW1_6803050474.json
Deleted EUW1_6806000027.json
Deleted EUW1_6774144258.json
Deleted EUW1_6780392475.json
Deleted EUW1_6801494379.json
Deleted EUW1_6810332743.jso

In [31]:
folder1="D:\EUW\CleanedGoldMatchJsons"
folder2="D:\EUW\CleanedGoldPostMatchDataJsons"
find_exclusive_files(folder1, folder2)

No files exclusive to D:\EUW\CleanedGoldMatchJsons
No files exclusive to D:\EUW\CleanedGoldPostMatchDataJsons


In [32]:
folder1="D:\EUW\CleanedPlatinumMatchJsons"
folder2="D:\EUW\CleanedPlatinumPostMatchDataJsons"
find_and_delete_exclusive_files(folder1, folder2)

Deleting files exclusive to D:\EUW\CleanedPlatinumMatchJsons:
Deleted EUW1_6791252063.json
Deleted EUW1_6808966258.json
Deleted EUW1_6801459698.json
Deleted EUW1_6786576795.json
Deleted EUW1_6810462789.json
Deleted EUW1_6790339288.json
Deleted EUW1_6817204370.json
Deleted EUW1_6815532148.json
Deleted EUW1_6790432560.json
Deleted EUW1_6793435242.json
Deleted EUW1_6807007202.json
Deleted EUW1_6778843383.json
Deleted EUW1_6806830623.json
Deleted EUW1_6796801839.json
Deleted EUW1_6799613484.json
Deleted EUW1_6794757840.json
Deleted EUW1_6806927580.json
Deleted EUW1_6806370376.json
Deleted EUW1_6790293028.json
Deleted EUW1_6817047813.json
Deleted EUW1_6814185144.json
Deleted EUW1_6812227840.json
Deleted EUW1_6798534624.json
Deleted EUW1_6791137782.json
Deleted EUW1_6793935968.json
Deleted EUW1_6798631319.json
Deleted EUW1_6795486709.json
Deleted EUW1_6807593044.json
Deleted EUW1_6816384169.json
Deleted EUW1_6809749100.json
Deleted EUW1_6790369664.json
Deleted EUW1_6806858462.json
Deleted EU

In [33]:
folder1="D:\EUW\CleanedPlatinumMatchJsons"
folder2="D:\EUW\CleanedPlatinumPostMatchDataJsons"
find_exclusive_files(folder1, folder2)

No files exclusive to D:\EUW\CleanedPlatinumMatchJsons
No files exclusive to D:\EUW\CleanedPlatinumPostMatchDataJsons


In [34]:
folder1="D:\EUW\CleanedEmeraldMatchJsons"
folder2="D:\EUW\CleanedEmeraldPostMatchDataJsons"
find_and_delete_exclusive_files(folder1, folder2)

No files exclusive to D:\EUW\CleanedEmeraldMatchJsons
Deleting files exclusive to D:\EUW\CleanedEmeraldPostMatchDataJsons:
Deleted EUW1_6810633717.json
Deleted EUW1_6814218866.json
Deleted EUW1_6807743577.json
Deleted EUW1_6789037911.json
Deleted EUW1_6808286727.json
Deleted EUW1_6798727666.json
Deleted EUW1_6807319047.json
Deleted EUW1_6792696774.json
Deleted EUW1_6808572122.json
Deleted EUW1_6804012055.json
Deleted EUW1_6792412189.json
Deleted EUW1_6793640551.json
Deleted EUW1_6805718200.json
Deleted EUW1_6801358672.json
Deleted EUW1_6802044308.json
Deleted EUW1_6814416743.json
Deleted EUW1_6815418494.json
Deleted EUW1_6809613854.json
Deleted EUW1_6798184996.json
Deleted EUW1_6803378247.json
Deleted EUW1_6792089595.json
Deleted EUW1_6803845321.json
Deleted EUW1_6802740208.json
Deleted EUW1_6789113859.json
Deleted EUW1_6817452372.json
Deleted EUW1_6810420323.json
Deleted EUW1_6817471244.json
Deleted EUW1_6798471047.json
Deleted EUW1_6809304401.json
Deleted EUW1_6812142629.json
Deleted

In [35]:
folder1="D:\EUW\CleanedEmeraldMatchJsons"
folder2="D:\EUW\CleanedEmeraldPostMatchDataJsons"
find_exclusive_files(folder1, folder2)

No files exclusive to D:\EUW\CleanedEmeraldMatchJsons
No files exclusive to D:\EUW\CleanedEmeraldPostMatchDataJsons


In [36]:
folder1="D:\EUW\CleanedDiamondMatchJsons"
folder2="D:\EUW\CleanedDiamondPostMatchDataJsons"
find_and_delete_exclusive_files(folder1, folder2)

Deleting files exclusive to D:\EUW\CleanedDiamondMatchJsons:
Deleted EUW1_6795887958.json
Deleted EUW1_6793378218.json
Deleted EUW1_6782024207.json
Deleted EUW1_6807759435.json
Deleted EUW1_6815893410.json
Deleted EUW1_6799694965.json
Deleted EUW1_6790178007.json
Deleted EUW1_6788518574.json
Deleted EUW1_6780550567.json
Deleted EUW1_6803443917.json
Deleted EUW1_6799568713.json
Deleted EUW1_6815906665.json
Deleted EUW1_6799537800.json
Deleted EUW1_6809487215.json
Deleted EUW1_6811238553.json
Deleted EUW1_6782169901.json
Deleted EUW1_6807647731.json
Deleted EUW1_6801845125.json
Deleted EUW1_6813066652.json
Total exclusive files deleted in D:\EUW\CleanedDiamondMatchJsons: 19
Deleting files exclusive to D:\EUW\CleanedDiamondPostMatchDataJsons:
Deleted EUW1_6814088190.json
Deleted EUW1_6807055504.json
Deleted EUW1_6781838868.json
Deleted EUW1_6798912353.json
Deleted EUW1_6805640663.json
Deleted EUW1_6800403127.json
Deleted EUW1_6778569907.json
Deleted EUW1_6779702603.json
Deleted EUW1_67974

In [37]:
folder1="D:\EUW\CleanedDiamondMatchJsons"
folder2="D:\EUW\CleanedDiamondPostMatchDataJsons"
find_exclusive_files(folder1, folder2)

No files exclusive to D:\EUW\CleanedDiamondMatchJsons
No files exclusive to D:\EUW\CleanedDiamondPostMatchDataJsons
